## Euroleague Fantasy Data Analytics Model

In [1]:
# pip install selenium webdriver-manager bs4 pandas lxml

import os, re, time, pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------------- helpers ----------------

def _clean_list(xs):
    return [re.sub(r"\s+\(.*?\)\s*$", "", x) for x in xs]

def _try_click_consent(driver, timeout=6):
    XPATHS = [
        "//button[contains(.,'Accept')]",
        "//button[contains(.,'I Agree')]",
        "//button[contains(.,'Agree')]",
        "//button[contains(.,'Αποδοχή')]",
        "//button[contains(.,'Συμφωνώ')]",
    ]
    end = time.time() + timeout
    for xp in XPATHS:
        try:
            btn = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, xp)))
            btn.click()
            return True
        except Exception:
            if time.time() > end: break
    return False

def _progress_scroll(driver, steps=10, pause=0.8):
    h = driver.execute_script("return document.body.scrollHeight || document.documentElement.scrollHeight;")
    for i in range(1, steps + 1):
        y = int(h * i / steps)
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(pause)

def _extract_team(side):
    team_el = side.select_one(".lineup__abbr, .lineup__team-name, .lineup__name")
    if team_el:
        return team_el.get_text(strip=True)
    logo = side.select_one("img[alt]")
    return (logo.get("alt") or "").strip() if logo else ""

def _extract_status(side):
    status_el = side.select_one(".lineup__status")
    txt = (status_el.get_text(" ", strip=True) if status_el else "").upper()
    if "CONFIRM" in txt:  return "CONFIRMED"
    if "EXPECT" in txt or "PROBABLE" in txt: return "EXPECTED"
    return "UNKNOWN"

def _extract_starters(side):
    # Try several variants for starters content
    containers = side.select(".lineup__list--starters, .lineup__list, .lineup__players")
    if not containers:
        containers = [side]

    names = []
    for blk in containers:
        for a in blk.select("a.lineup__player-link, .lineup__player a"):
            t = a.get_text(" ", strip=True)
            if t: names.append(t)
        if not names:
            for row in blk.select(".lineup__player"):
                t = row.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)
        if not names:
            for li in blk.select("li"):
                t = li.get_text(" ", strip=True)
                if re.match(r"^(PG|SG|SF|PF|C)\b", t): names.append(t)

    if not names:
        txt = side.get_text("\n", strip=True)
        names = re.findall(r"(?:^|\n)(?:PG|SG|SF|PF|C)\s+[^\n]+", txt)

    return _clean_list(names)[:5]

# ---------------- main ----------------

def fetch_rotowire_lineups_selenium(date: str | None = None,
                                    wait_sec: float = 14.0,
                                    headless: bool = False) -> pd.DataFrame:
    """
    Render Rotowire lineups & parse BOTH sides per game (global side selectors).
    Returns:
      game_time, team, side (AWAY/HOME), lineup_status, starters,
      starter_1..starter_5, lineup_confirmed (0/1)
    """
    base = "https://www.rotowire.com/euro/daily-lineups.php"
    url = base if not date else f"{base}?date={date}"

    opts = Options()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1400,1000")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--lang=en-US,en;q=0.9")
    opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    driver.get(url)

    _try_click_consent(driver, timeout=6)
    time.sleep(1.2)
    try:
        WebDriverWait(driver, int(wait_sec)).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".lineup, .lineup.is-nba"))
        )
    except Exception:
        pass

    _progress_scroll(driver, steps=10, pause=0.8)
    time.sleep(1.0)

    # quick diagnostics
    blocks = driver.find_elements(By.CSS_SELECTOR, ".lineup.is-nba, .lineup")
    players = driver.find_elements(By.CSS_SELECTOR, ".lineup__player, a.lineup__player-link")
    print(f"diagnostics: lineup blocks={len(blocks)}, player nodes={len(players)}")

    html = driver.page_source
    os.makedirs("_rotowire_debug", exist_ok=True)
    with open("_rotowire_debug/last_lineups.html", "w", encoding="utf-8") as f:
        f.write(html)
    try:
        driver.save_screenshot("_rotowire_debug/last_lineups.png")
    except Exception:
        pass
    driver.quit()

    # -------- parse globally by side classes ----------
    soup = BeautifulSoup(html, "lxml")

    # game time map: find each game container time
    game_time_map = {}
    for gi, g in enumerate(soup.select(".lineup__main, .lineup.is-nba, .lineup")):
        t = g.select_one(".lineup__time, .game-time")
        game_time_map[id(g)] = t.get_text(strip=True) if t else ""

    # Select **visit/away** & **home** side boxes explicitly
    visit_sel = (
        '[class*="lineup__box"][class*="is-visit"], '
        '[class*="lineup__team"][class*="is-visit"], '
        '[class*="lineup__side"][class*="is-visit"], '
        '[class*="visit"]'
    )
    home_sel = (
        '[class*="lineup__box"][class*="is-home"], '
        '[class*="lineup__team"][class*="is-home"], '
        '[class*="lineup__side"][class*="is-home"], '
        '[class*="home"]'
    )

    visit_boxes = soup.select(visit_sel)
    home_boxes  = soup.select(home_sel)

    rows = []

    def add_rows(boxes, side_label):
        for box in boxes:
            # nearest parent game container for time
            parent = box.find_parent(lambda tag: tag.has_attr("class") and any(
                c in {"lineup__main","lineup","lineup is-nba"} for c in tag.get("class", [])
            ))
            game_time = game_time_map.get(id(parent), "") if parent else ""
            team = _extract_team(box)
            starters = _extract_starters(box)
            status = _extract_status(box)
            if starters or team:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side_label,
                    "lineup_status": status,
                    "starters": starters,
                    "starter_1": starters[0] if len(starters)>0 else None,
                    "starter_2": starters[1] if len(starters)>1 else None,
                    "starter_3": starters[2] if len(starters)>2 else None,
                    "starter_4": starters[3] if len(starters)>3 else None,
                    "starter_5": starters[4] if len(starters)>4 else None,
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    add_rows(visit_boxes, "AWAY")
    add_rows(home_boxes,  "HOME")

    df = pd.DataFrame(rows)

    if not df.empty:
        df = df.drop_duplicates(
            subset=["game_time","team","side","starter_1","starter_2","starter_3","starter_4","starter_5"]
        )
        all_na = df[["starter_1","starter_2","starter_3","starter_4","starter_5"]].isna().all(axis=1)
        df = df[~all_na].reset_index(drop=True)
    else:
        print("⚠️ Parsed zero rows. Check _rotowire_debug/last_lineups.html & .png")

    return df


# ---------- run it ----------
df_lineups = fetch_rotowire_lineups_selenium(wait_sec=14.0, headless=False)
print("✅ Shape:", df_lineups.shape)
print(df_lineups.sort_values(["game_time","side"]).head(12).to_string(index=False))


diagnostics: lineup blocks=12, player nodes=168
✅ Shape: (20, 11)
game_time team side lineup_status                                                                 starters      starter_1        starter_2     starter_3     starter_4     starter_5  lineup_confirmed
               AWAY     CONFIRMED       [J. Robinson, S. Herrera, Jared Rhoden, Derek Willis, Ismael Bako]    J. Robinson       S. Herrera  Jared Rhoden  Derek Willis   Ismael Bako                 1
               AWAY     CONFIRMED        [Luca Vildoza, C. Edwards, Nicola Akele, D. Alston Jr., M. Diouf]   Luca Vildoza       C. Edwards  Nicola Akele D. Alston Jr.      M. Diouf                 1
               AWAY     CONFIRMED          [Kobi Simmons, Rafa Villar, H. Diallo, R. Kurucs, Khalifa Diop]   Kobi Simmons      Rafa Villar     H. Diallo     R. Kurucs  Khalifa Diop                 1
               AWAY     CONFIRMED         [F. Campazzo, A. Abalde, Mario Hezonja, Chuma Okeke, W. Tavares]    F. Campazzo        A. Abalde

In [2]:
# -- Cell 2: parse saved HTML to starters + MNP count --------------------------
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Folders (fallbacks, in case Cell 1 wasn't run)
DATA_DIR = "data_raw"; DEBUG_DIR = "_rotowire_debug"
os.makedirs(DATA_DIR, exist_ok=True); os.makedirs(DEBUG_DIR, exist_ok=True)

HTML_PATH = f"{DEBUG_DIR}/last_lineups.html" if os.path.exists(f"{DEBUG_DIR}/last_lineups.html") else "last_lineups.html"
def _txt(x):
    return re.sub(r"\s+", " ", x.get_text(" ", strip=True)) if x else ""

def _clean_player(n):
    if not n:
        return n
    n = re.sub(r"\s+\(.*?\)\s*$", "", n).strip()
    n = re.sub(r"^(PG|SG|SF|PF|C)\s+", "", n, flags=re.I)
    return n

def _get_mnp_from_ul(ul):
    """Extract 'May Not Play' entries from a team UL."""
    mnp = []
    title = ul.find("li", class_=lambda c: c and "lineup__title" in c and re.search(
        r"may\s+not\s+play", _txt(ul.find("li", class_=c)) if ul.find("li", class_=c) else "", re.I
    ))
    if title:
        for li in title.find_all_next("li"):
            if "lineup__title" in (li.get("class") or []):
                break
            if "lineup__player" in (li.get("class") or []):
                a = li.select_one("a")
                tag = li.select_one(".lineup__inj")
                nm = _txt(a) if a else ""
                if nm:
                    mnp.append(f"{nm} ({_txt(tag)})" if tag else nm)
        return [_clean_player(x) for x in mnp if x and x.lower() != "none"]

    for li in ul.select(".lineup__notplay li, .lineup__status--out, .lineup__inj-list li"):
        nm = _txt(li)
        if nm:
            mnp.append(_clean_player(nm))
    return [x for x in mnp if x and x.lower() != "none"]

def _extract_starters_from_ul(ul):
    names = []
    for li in ul.select("li.lineup__player.is-pct-play-100 a"):
        nm = _txt(li)
        if nm:
            names.append(nm)
    if len(names) < 5:
        for li in ul.select("li.lineup__player a"):
            nm = _txt(li)
            if nm:
                names.append(nm)
            if len(names) >= 5:
                break
    names = [_clean_player(n) for n in names]
    return names[:5]

def _lineup_status(ul):
    st = _txt(ul.select_one(".lineup__status"))
    stU = st.upper()
    if "CONFIRM" in stU: return "CONFIRMED"
    if "EXPECT" in stU or "PROBABLE" in stU: return "EXPECTED"
    return "UNKNOWN"

def parse_rotowire_lineups_flexible(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        html = f.read()
    soup = BeautifulSoup(html, "lxml")

    diag = {
        "lineup__teams": len(soup.select("div.lineup__teams")),
        "ul.lineup__list": len(soup.select("ul.lineup__list")),
        "ul.is-visit": len(soup.select("ul.lineup__list.is-visit")),
        "ul.is-home": len(soup.select("ul.lineup__list.is-home")),
        "see-proj-minutes buttons": len(soup.select("button.see-proj-minutes")),
        "header abbr": len(soup.select(".lineup__hdr .lineup__abbr")),
        "header team": len(soup.select(".lineup__hdr .lineup__team")),
        "player anchors": len(soup.select("a.lineup__player-link, .lineup__player a")),
        "MNP titles": len(soup.find_all(string=re.compile(r"^\s*may\s+not\s+play\s*$", re.I))),
    }
    print("DOM diagnostics:", diag)

    rows = []

    # Strategy A: by matchup blocks
    for teams_div in soup.select("div.lineup__teams"):
        time_el = teams_div.find_previous("div", class_="lineup__time")
        game_time = _txt(time_el)

        uls = teams_div.select("ul.lineup__list")
        if len(uls) < 1:
            continue

        away_ul = None
        home_ul = None
        for ul in uls:
            classes = " ".join(ul.get("class", [])).lower()
            if "is-visit" in classes or "visit" in classes or "away" in classes:
                away_ul = ul
            if "is-home" in classes or "home" in classes:
                home_ul = home_ul or ul

        if away_ul is None and home_ul is None and len(uls) >= 2:
            away_ul, home_ul = uls[0], uls[1]
        elif away_ul is None and len(uls) >= 1:
            away_ul = uls[0]
        elif home_ul is None and len(uls) >= 2:
            home_ul = next((u for u in uls if u is not away_ul), None)

        header_abbrs = [_txt(el) for el in teams_div.select(".lineup__abbr") if _txt(el)]
        if not header_abbrs:
            parent_main = teams_div.find_parent(["div","section"])
            if parent_main:
                header_abbrs = [_txt(el) for el in parent_main.select(".lineup__abbr") if _txt(el)]

        for idx, (side, ul) in enumerate([("AWAY", away_ul), ("HOME", home_ul)]):
            if not ul:
                continue
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            if not team and header_abbrs and idx < len(header_abbrs):
                team = header_abbrs[idx].upper()

            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            if team or starters or mnp:
                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    # Strategy B: global scan if A found nothing
    if not rows:
        print("Fallback B: scanning all ul.lineup__list globally...")
        for ul in soup.select("ul.lineup__list"):
            side = "AWAY" if "is-visit" in (ul.get("class") or []) else ("HOME" if "is-home" in (ul.get("class") or []) else None)
            btn = ul.select_one("button.see-proj-minutes")
            team = btn["data-team"].strip().upper() if btn and btn.has_attr("data-team") else None
            starters = _extract_starters_from_ul(ul)
            mnp = _get_mnp_from_ul(ul)
            status = _lineup_status(ul)

            if side and (team or starters or mnp):
                rows.append({
                    "game_time": "",
                    "team": team,
                    "side": side,
                    "lineup_status": status,
                    "starters": starters,
                    "may_not_play": mnp,
                    "may_not_play_count": len(mnp),
                    "lineup_confirmed": int(status == "CONFIRMED"),
                })

    df = pd.DataFrame(rows)
    for i in range(5):
        col = f"starter_{i+1}"
        if "starters" in df.columns:
            df[col] = df["starters"].apply(lambda xs: xs[i] if isinstance(xs, list) and len(xs) > i else None)

    print(f"→ Parsed rows: {len(df)}")

    # Save a copy for downstream
    stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    out_path = f"{DATA_DIR}/lineups_parsed_{stamp}.csv"
    df.to_csv(out_path, index=False)
    print(f"💾 Saved parsed lineups to {out_path}")

    return df

# ---- RUN IT (point to the saved HTML) ----
HTML_PATH = f"{DEBUG_DIR}/last_lineups.html"
if not os.path.exists(HTML_PATH) and os.path.exists("last_lineups.html"):
    HTML_PATH = "last_lineups.html"

df_lineups = parse_rotowire_lineups_flexible(HTML_PATH)

if df_lineups.empty:
    print("\n⚠️ Still empty. Check DOM diagnostics and ensure Cell 3 ran successfully.")
else:
    cols = ["game_time","team","side","lineup_status","may_not_play_count",
            "starter_1","starter_2","starter_3","starter_4","starter_5"]
    print("\n✅ Preview:")
    print(df_lineups[cols].sort_values(["game_time","side","team"], na_position="last").to_string(index=False))


DOM diagnostics: {'lineup__teams': 10, 'ul.lineup__list': 20, 'ul.is-visit': 10, 'ul.is-home': 10, 'see-proj-minutes buttons': 0, 'header abbr': 0, 'header team': 0, 'player anchors': 168, 'MNP titles': 0}
Fallback B: scanning all ul.lineup__list globally...
→ Parsed rows: 20
💾 Saved parsed lineups to data_raw/lineups_parsed_20251126_145838.csv

✅ Preview:
game_time team side lineup_status  may_not_play_count          starter_1        starter_2     starter_3      starter_4     starter_5
          None AWAY     CONFIRMED                   0        J. Robinson       S. Herrera  Jared Rhoden   Derek Willis   Ismael Bako
          None AWAY     CONFIRMED                   0       Luca Vildoza       C. Edwards  Nicola Akele  D. Alston Jr.      M. Diouf
          None AWAY     CONFIRMED                   0       Kobi Simmons      Rafa Villar     H. Diallo      R. Kurucs  Khalifa Diop
          None AWAY     CONFIRMED                   0        F. Campazzo        A. Abalde Mario Hezonja    Ch

In [3]:
# ============================================
# Cell #5 — Safe coalesce & name builder utils (RUN FIRST)
# ============================================
import pandas as pd
import numpy as np

def coalesce(df, keys, numeric=False):
    """
    Return the first non-null column among the list of keys in df.
    Keeps alignment with df.index and handles both text & numeric safely.
    """
    out = None
    for k in keys:
        if k in df.columns:
            s = df[k]
            if numeric:
                s = pd.to_numeric(s, errors="coerce")
            out = s if out is None else out.where(out.notna(), s)
    if out is None:
        return pd.Series([np.nan] * len(df), index=df.index)
    return out

def build_name_short(df):
    """
    Build a player name string like 'F. Lastname' or use full name if present.
    Works even when first/last fields are missing.
    """
    def _as_text(s):
        if s is None or not isinstance(s, pd.Series):
            return pd.Series(pd.array([""] * len(df), dtype="string"))
        return s.astype("string").fillna("")

    # Prefer a ready-made full name first
    full = _as_text(coalesce(df, [
        "player_name", "player.name", "name",
        "displayName", "shortName"
    ]))
    has_full = full.fillna("").ne("")

    # Fallback: build from first + last
    first = _as_text(coalesce(df, [
        "player.first_name", "first_name", "firstname", "firstName"
    ]))
    last = _as_text(coalesce(df, [
        "player.last_name", "last_name", "surname", "lastName"
    ]))
    short = first.str.slice(0, 1).fillna("").str.upper() + ". " + last.fillna("")
    short = short.str.replace(r"^\.\s*$", "", regex=True).str.strip()

    # Prefer full when available
    out = pd.Series(pd.array([""] * len(df), dtype="string"))
    out[has_full] = full[has_full]
    out = out.mask(out.fillna("").eq(""), short)
    return out.fillna("")

# =======================================================
# Cell #7 — Build training set with Usage, DvP, and MNP (RUN SECOND)
# =======================================================
import pandas as pd, numpy as np, requests

SEASON_ID = 23
PREDICT_WEEK = 8

# Fetch data
STATS_URL = (
    "https://www.dunkest.com/api/stats/table?"
    "season_id=23&mode=dunkest&stats_type=tot&"
    "weeks[]=8&weeks[]=7&weeks[]=6&weeks[]=5&weeks[]=4&weeks[]=3&weeks[]=2&weeks[]=1&"
    "sort_by=pdk&sort_order=desc&iframe=yes"
)
headers = {"Accept": "application/json", "Referer": "https://www.dunkest.com/en/euroleague/stats/players/table"}
r = requests.get(STATS_URL, headers=headers, timeout=30)
r.raise_for_status()
records = r.json() if isinstance(r.json(), list) else r.json().get("data", [])
df = pd.json_normalize(records, sep=".")
print(f"✅ Downloaded {len(df)} player rows")

# Core fields (using functions from Cell 5)
df["Player"] = build_name_short(df)
df["PlayerID"] = coalesce(df, ["player.id", "id"], numeric=True)
df["Pos"] = coalesce(df, ["position_short", "position", "role"])
df["Team"] = coalesce(df, ["team_short", "team.code", "team.name"])
df["FPT"] = coalesce(df, ["pdk", "fpt", "fantasy_points", "avg", "points"], numeric=True)
df["CR"] = coalesce(df, ["cr", "credits", "price", "value"], numeric=True)
df["MIN"] = coalesce(df, ["min", "minutes"], numeric=True)

# Usage proxy
df["PTS"] = coalesce(df, ["pts"], numeric=True)
df["AST"] = coalesce(df, ["ast"], numeric=True)
df["TOV"] = coalesce(df, ["tov"], numeric=True)
df["FGA"] = coalesce(df, ["fga"], numeric=True)
df["FTA"] = coalesce(df, ["fta"], numeric=True)
# FIXED: Changed ["PTS"] to df["PTS"]
df["Usage_Raw"] = (df["PTS"] + df["AST"] + df["TOV"] + df["FGA"] + df["FTA"]) / 100.0
df["Usage_Per_Min"] = df["Usage_Raw"] 

# Injury/Missing flag from mnp_df
if "mnp_df" in globals() and not mnp_df.empty:
    out_names = mnp_df["player"].dropna().str.strip().str.lower().unique()
    df["Is_Out"] = df["Player"].str.lower().isin(out_names).astype(int)
else:
    df["Is_Out"] = 0

# Build dataset
train_df = df[["Player", "PlayerID", "Pos", "Team", "FPT", "CR", "MIN", "Usage_Raw", "Is_Out"]].copy()

# Add Allow_FPT (DvP)
if "dvp_df" in globals() and not dvp_df.empty:
    pos_allow = dvp_df.groupby("Pos")[["Allow_FPT"]].mean().reset_index()
else:
    pos_allow = train_df.groupby("Pos")[["FPT"]].mean().rename(columns={"FPT": "Allow_FPT"}).reset_index()
train_df = train_df.merge(pos_allow, on="Pos", how="left")

# Rolling means
train_df["FPT_roll_mean"] = train_df.groupby("Pos")["FPT"].transform(lambda s: s.rolling(3, min_periods=1).mean())
train_df["MIN_roll_mean"] = train_df.groupby("Pos")["MIN"].transform(lambda s: s.rolling(3, min_periods=1).mean())

# Clean numeric fields
for col in ["FPT", "CR", "MIN", "Allow_FPT", "FPT_roll_mean", "MIN_roll_mean", "Usage_Raw"]:
    train_df[col] = pd.to_numeric(train_df[col], errors="coerce")

train_df = train_df.dropna(subset=["FPT", "CR", "MIN"])
features = ["FPT_roll_mean", "MIN_roll_mean", "Allow_FPT", "CR", "Usage_Raw", "Is_Out"]

X = train_df[features].fillna(0.0)
y = train_df["FPT"].astype(float)

print(f"\n✅ Training samples: {len(X)} | Target: current FPT | Features: {features}")
display(X.head(10))
display(y.head(10))

# =======================================================
# Cell #8 — Train Model (FIXED - removed opponent logic)
# =======================================================
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Use features without opponent data (since schedule_df doesn't exist)
feature_cols = ["FPT_roll_mean", "MIN_roll_mean", "Allow_FPT", "CR", "Usage_Raw", "Is_Out"]
train_df[feature_cols] = train_df[feature_cols].apply(pd.to_numeric, errors="coerce")
train_df = train_df.dropna(subset=feature_cols + ["FPT"])

X = train_df[feature_cols].fillna(0.0)
y = train_df["FPT"].astype(float)

# Train model
model = RandomForestRegressor(n_estimators=300, max_depth=8, random_state=42)
model.fit(X, y)

# Evaluate performance
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)

print(f"✅ Model trained on {len(X)} samples")
print(f"   R² = {r2:.3f}   |   MAE = {mae:.2f} fantasy points")

# Export rolling & usage features per player
export_cols = ["PlayerID", "FPT_roll_mean", "MIN_roll_mean", "Usage_Raw"]
if "PlayerID" in train_df.columns:
    latest_roll = (
        train_df
        .sort_values("PlayerID")
        .groupby("PlayerID", group_keys=False)
        .tail(1)[export_cols]
        .reset_index(drop=True)
    )
else:
    latest_roll = train_df[[c for c in export_cols if c in train_df.columns]].copy()

print(f"💾 Stored rolling features for {len(latest_roll)} players.")
display(latest_roll.head(10))

✅ Downloaded 280 player rows

✅ Training samples: 280 | Target: current FPT | Features: ['FPT_roll_mean', 'MIN_roll_mean', 'Allow_FPT', 'CR', 'Usage_Raw', 'Is_Out']


,FPT_roll_mean,MIN_roll_mean,Allow_FPT,CR,Usage_Raw,Is_Out
0,132.700000,220.000000,51.1675,11.5,3.06,0
1,68.700000,128.500000,51.1675,4.0,0.08,0
2,71.800000,151.666667,51.1675,11.4,2.29,0
3,50.733333,134.333333,51.1675,7.0,1.54,0
4,99.366667,201.333333,51.1675,16.5,3.37,0
5,81.733333,165.000000,51.1675,6.3,0.81,0
6,61.000000,133.000000,51.1675,4.0,0.45,0
7,41.866667,116.666667,51.1675,10.9,2.36,0
8,35.666667,112.000000,51.1675,4.0,0.83,0
9,72.733333,133.333333,51.1675,12.6,2.22,0


0    132.7
1      4.7
2     78.0
3     69.5
4    150.6
5     25.1
6      7.3
7     93.2
8      6.5
9    118.5
Name: FPT, dtype: float64

✅ Model trained on 280 samples
   R² = 0.980   |   MAE = 5.00 fantasy points
💾 Stored rolling features for 280 players.


,PlayerID,FPT_roll_mean,MIN_roll_mean,Usage_Raw
0,1176,55.033333,124.000000,1.77
1,1177,132.700000,220.000000,3.06
2,1189,91.833333,181.333333,0.29
3,1195,103.033333,183.333333,2.67
4,1201,6.933333,37.666667,0.11
5,1206,75.866667,151.000000,3.23
6,1209,85.200000,141.666667,0.86
7,1210,99.533333,182.000000,2.80
8,1213,111.966667,174.000000,2.48
9,1214,54.600000,145.333333,1.22


In [4]:
# -------------------------------------------------------
# 6) Defense vs Position (DvP) — tolerant + normalized
# -------------------------------------------------------
import requests, pandas as pd, numpy as np

def _first_records(obj):
    """Find the first list[dict] inside a Dunkest JSON payload."""
    if isinstance(obj, list) and (len(obj) == 0 or isinstance(obj[0], dict)):
        return obj
    if isinstance(obj, dict):
        for v in obj.values():
            rec = _first_records(v)
            if isinstance(rec, list) and (len(rec) == 0 or isinstance(rec[0], dict)):
                return rec
    return []

def _coalesce(df, keys, numeric=False):
    """Coalesce multiple possible columns into one unified Series."""
    out = None
    for k in keys:
        if k in df.columns:
            s = df[k]
            s = pd.to_numeric(s, errors="coerce") if numeric else s
            out = s if out is None else out.where(out.notna(), s)
    if out is None:
        out = pd.Series([np.nan] * len(df))
    return out

def fetch_dvp_all_positions(season_id=23, stats_id=25, position_ids=(1, 2, 3), timeout=30):
    """
    Fetch DvP (Defense vs Position) for G/F/C from Dunkest.
    Returns Team, Pos, Allow_FPT + L3/L5/L10, Rank, GP.
    """
    base = "https://www.dunkest.com/api/stats/defense-vs-position"
    pos_map = {1: "G", 2: "F", 3: "C"}
    frames = []

    for pid in position_ids:
        params = {"season_id": season_id, "stats_id": stats_id, "position_id": pid}
        try:
            r = requests.get(base, params=params, headers={"Accept": "application/json"}, timeout=timeout)
            r.raise_for_status()
            js = r.json()
        except Exception as e:
            print(f"⚠️ DvP fetch failed for position {pid}: {e}")
            continue

        rec = _first_records(js)
        raw = pd.json_normalize(rec, sep=".")
        if raw.empty:
            print(f"⚠️ No records for position {pid}")
            continue

        # Accept most likely column variants
        team = _coalesce(raw, ["team_short", "team.code", "team_name", "team.name", "team", "name"])
        allow = _coalesce(raw, ["pdk_allowed", "avg_pdk", "fantasy_points_allowed",
                                "avg", "pdk", "fpt_allowed"], numeric=True)
        l3 = _coalesce(raw, ["l3", "L3"], numeric=True)
        l5 = _coalesce(raw, ["l5", "L5"], numeric=True)
        l10 = _coalesce(raw, ["l10", "L10"], numeric=True)
        rank = _coalesce(raw, ["rank", "ranking"], numeric=True)
        gp = _coalesce(raw, ["gp", "games"], numeric=True)

        tidy = pd.DataFrame({
            "Team": team,
            "Pos": pos_map.get(pid, str(pid)),
            "Allow_FPT": allow,
            "L3": l3,
            "L5": l5,
            "L10": l10,
            "Rank": rank,
            "GP": gp,
        })

        if tidy["Team"].isna().any() and "id" in raw.columns:
            tidy.loc[tidy["Team"].isna(), "Team"] = raw.loc[tidy["Team"].isna(), "id"].astype(str)

        frames.append(tidy)

    if not frames:
        print("⚠️ No DvP data found for any position — returning empty DataFrame.")
        return pd.DataFrame(columns=["Team", "Pos", "Allow_FPT", "L3", "L5", "L10", "Rank", "GP"])

    dvp = pd.concat(frames, ignore_index=True)
    dvp["Team"] = dvp["Team"].astype(str).str.strip()
    dvp = dvp.drop_duplicates(subset=["Team", "Pos"]).reset_index(drop=True)

    # Fill Allow_FPT from L5/L3 if missing
    if dvp["Allow_FPT"].isna().all():
        for fallback in ["L5", "L3", "L10"]:
            if fallback in dvp.columns and dvp[fallback].notna().any():
                dvp["Allow_FPT"] = dvp[fallback]
                break

    dvp = dvp.sort_values(["Pos", "Allow_FPT"], ascending=[True, True]).reset_index(drop=True)
    return dvp


# === Fetch and preview ===
dvp_df = fetch_dvp_all_positions(season_id=23, stats_id=25)
print(f"✅ DvP rows fetched: {len(dvp_df)}")
display(dvp_df.head(12))
display(dvp_df[dvp_df["Pos"] == "G"].head(10))


✅ DvP rows fetched: 60


,Team,Pos,Allow_FPT,L3,L5,L10,Rank,GP
0,AS Monaco,C,7.04,6.30,7.04,12.39,NaN,NaN
1,Baskonia Vitoria-Gasteiz,C,8.10,0.00,8.10,14.23,NaN,NaN
2,FC Bayern Munich,C,10.24,8.53,10.24,15.05,NaN,NaN
3,Hapoel IBI Tel Aviv,C,10.45,8.80,10.45,11.69,NaN,NaN
4,Maccabi Rapyd Tel Aviv,C,10.95,10.17,10.95,16.08,NaN,NaN
5,Olympiacos Piraeus,C,11.56,13.40,11.56,9.91,NaN,NaN
6,Fenerbahce Beko Istanbul,C,12.75,14.50,12.75,15.08,NaN,NaN
7,Virtus Bologna,C,13.03,14.80,13.03,14.04,NaN,NaN
8,Dubai Basketball,C,14.03,16.65,14.03,11.11,NaN,NaN
9,Valencia Basket,C,14.37,13.20,14.37,14.31,NaN,NaN


,Team,Pos,Allow_FPT,L3,L5,L10,Rank,GP
40,Fenerbahce Beko Istanbul,G,8.42,9.75,8.42,11.38,NaN,NaN
41,Olympiacos Piraeus,G,11.34,14.38,11.34,12.08,NaN,NaN
42,Hapoel IBI Tel Aviv,G,11.42,12.05,11.42,12.29,NaN,NaN
43,Virtus Bologna,G,12.11,11.60,12.11,12.84,NaN,NaN
44,Panathinaikos AKTOR Athens,G,12.15,11.20,12.15,12.97,NaN,NaN
45,Crvena Zvezda Meridianbet Belgrade,G,12.26,13.97,12.26,11.49,NaN,NaN
46,Anadolu Efes Istanbul,G,12.53,11.75,12.53,11.66,NaN,NaN
47,AS Monaco,G,12.60,14.12,12.60,13.24,NaN,NaN
48,FC Bayern Munich,G,13.12,13.93,13.12,10.91,NaN,NaN
49,Valencia Basket,G,13.14,10.54,13.14,14.84,NaN,NaN


In [5]:
# -- Cell 3: Parse "May Not Play" (MNP) from saved HTML -----------------------
# pip install bs4 lxml pandas
import os, re, pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Folders (fallbacks, in case Cell 1 wasn't run)
DATA_DIR = "data_raw"; DEBUG_DIR = "_rotowire_debug"
os.makedirs(DATA_DIR, exist_ok=True); os.makedirs(DEBUG_DIR, exist_ok=True)

HTML_PATH = f"{DEBUG_DIR}/last_lineups.html" if os.path.exists(f"{DEBUG_DIR}/last_lineups.html") else "last_lineups.html"

LIKELIHOOD_MAP = {
    "is-pct-play-100": 100, "is-pct-play-90": 90, "is-pct-play-75": 75,
    "is-pct-play-60": 60, "is-pct-play-50": 50, "is-pct-play-40": 40,
    "is-pct-play-25": 25, "is-pct-play-10": 10, "is-pct-play-0": 0
}

def _txt(node): 
    return re.sub(r"\s+", " ", node.get_text(" ", strip=True)) if node else ""

def _likelihood_from_classes(classes):
    for c in classes or []:
        if c in LIKELIHOOD_MAP:
            return LIKELIHOOD_MAP[c]
    return None

def _clean_player(n):
    if not n: return n
    n = re.sub(r"\s+\(.*?\)\s*$", "", n).strip()
    n = re.sub(r"^(PG|SG|SF|PF|C)\s+", "", n, flags=re.I)
    return n

def parse_rotowire_mnp_final(html_path: str) -> pd.DataFrame:
    with open(html_path, "r", encoding="utf-8", errors="ignore") as f:
        soup = BeautifulSoup(f.read(), "lxml")

    rows = []

    # Primary structure
    games = soup.select("div.lineup")  # More general for Euroleague

    print(f"Found {len(games)} games in HTML.")

    for game in games:
        game_time = _txt(game.select_one(".lineup__time"))
        # Pair teams by .lineup__team, then iterate their ULs
        team_blocks = game.select(".lineup__team")
        teams = []
        for tb in team_blocks:
            abbr = _txt(tb.select_one(".lineup__abbr")) or _txt(tb.select_one(".lineup__team-name"))
            side = "AWAY" if "is-visit" in (tb.get("class") or []) else ("HOME" if "is-home" in (tb.get("class") or []) else None)
            teams.append((abbr, side))

        ul_lists = game.select("ul.lineup__list")
        for idx, ul in enumerate(ul_lists):
            team, side = (teams[idx] if idx < len(teams) else (None, None))
            # Find the MNP title in this UL
            mnp_title = ul.find("li", class_="lineup__title", string=lambda s: s and "MAY NOT PLAY" in s.upper())
            if not mnp_title:
                continue

            for li in mnp_title.find_next_siblings("li"):
                classes = li.get("class") or []
                if "lineup__title" in classes:
                    break
                if "lineup__player" not in classes:
                    continue

                pos = _txt(li.select_one(".lineup__pos"))
                a = li.select_one("a")
                player = _clean_player(_txt(a))
                if not player:
                    continue

                status = _txt(li.select_one(".lineup__inj"))
                title_text = (li.get("title") or "").strip()
                likelihood_pct = _likelihood_from_classes(classes)

                rows.append({
                    "game_time": game_time,
                    "team": team,
                    "side": side,
                    "position": pos,
                    "player": player,
                    "status": status,
                    "title_text": title_text,
                    "likelihood_pct": likelihood_pct
                })

    # Fallback: global scan (if nothing found in primary structure)
    if not rows:
        print("Fallback: global MNP scan…")
        for ul in soup.select("ul.lineup__list"):
            for li in ul.select("li.lineup__player"):
                inj = li.select_one(".lineup__inj")
                if not inj:
                    continue
                a = li.select_one("a")
                player = _clean_player(_txt(a))
                if not player:
                    continue
                rows.append({
                    "game_time": "",
                    "team": None,
                    "side": None,
                    "position": _txt(li.select_one(".lineup__pos")),
                    "player": player,
                    "status": _txt(inj),
                    "title_text": (li.get("title") or "").strip(),
                    "likelihood_pct": None
                })


    df = pd.DataFrame(rows)
    if df.empty:
        print("⚠️ No 'May Not Play' players found. Check if Rotowire changed markup or re-run Cell 3.")
        return df

    df = df.sort_values(["game_time","side","team","player"], na_position="last").reset_index(drop=True)
    print(f"✅ Parsed {len(df)} 'May Not Play' players across {df['team'].nunique(dropna=True)} teams.")
    return df

# ---- RUN ----
mnp_df = parse_rotowire_mnp_final(HTML_PATH)
if not mnp_df.empty:
    print(mnp_df.head(30).to_string(index=False))
    stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    out_csv = f"{DATA_DIR}/may_not_play_players_{stamp}.csv"
    mnp_df.to_csv(out_csv, index=False)
    print(f"\n💾 Saved: {out_csv}")


Found 12 games in HTML.
Fallback: global MNP scan…
✅ Parsed 69 'May Not Play' players across 0 teams.
game_time team side position         player status title_text likelihood_pct
          None None        G   A. Avramovic    OUT                      None
          None None        F A. Butkevicius    OUT                      None
          None None        F   A. Smailagic    OUT                      None
          None None        F   Abramo Canka    OUT                      None
          None None        C       Alex Len    GTD                      None
          None None        F   Arijan Lakic    OUT                      None
          None None        G      B. Taylor    OUT                      None
          None None        F    Braxton Key    GTD                      None
          None None        F  Bruno Caboclo    OUT                      None
          None None        G   Carlik Jones    OUT                      None
          None None        F     Cedi Osman    OUT 

In [6]:
# =======================================================
# Cell #7 — Build training set with Usage, DvP, and MNP
# =======================================================
import pandas as pd, numpy as np, requests

SEASON_ID = 23
PREDICT_WEEK = 8

# Fetch data
STATS_URL = (
    "https://www.dunkest.com/api/stats/table?"
    "season_id=23&mode=dunkest&stats_type=tot&"
    "weeks[]=8&weeks[]=7&weeks[]=6&weeks[]=5&weeks[]=4&weeks[]=3&weeks[]=2&weeks[]=1&"
    "sort_by=pdk&sort_order=desc&iframe=yes"
)
headers = {"Accept": "application/json", "Referer": "https://www.dunkest.com/en/euroleague/stats/players/table"}
r = requests.get(STATS_URL, headers=headers, timeout=30)
r.raise_for_status()
records = r.json() if isinstance(r.json(), list) else r.json().get("data", [])
df = pd.json_normalize(records, sep=".")
print(f"✅ Downloaded {len(df)} player rows")

# Core fields
df["Player"] = build_name_short(df)
df["PlayerID"] = coalesce(df, ["player.id", "id"], numeric=True)
df["Pos"] = coalesce(df, ["position_short", "position", "role"])
df["Team"] = coalesce(df, ["team_short", "team.code", "team.name"])
df["FPT"] = coalesce(df, ["pdk", "fpt", "fantasy_points", "avg", "points"], numeric=True)
df["CR"] = coalesce(df, ["cr", "credits", "price", "value"], numeric=True)
df["MIN"] = coalesce(df, ["min", "minutes"], numeric=True)

# Usage proxy
df["PTS"] = coalesce(df, ["pts"], numeric=True)
df["AST"] = coalesce(df, ["ast"], numeric=True)
df["TOV"] = coalesce(df, ["tov"], numeric=True)
df["FGA"] = coalesce(df, ["fga"], numeric=True)
df["FTA"] = coalesce(df, ["fta"], numeric=True)
df["Usage_Raw"] = (df["PTS"] + df["AST"] + df["TOV"] + df["FGA"] + df["FTA"]) / 100.0

# Injury/Missing flag from mnp_df
if "mnp_df" in globals() and not mnp_df.empty:
    out_names = mnp_df["player"].dropna().str.strip().str.lower().unique()
    df["Is_Out"] = df["Player"].str.lower().isin(out_names).astype(int)
else:
    df["Is_Out"] = 0

# Build dataset
train_df = df[["Player", "PlayerID", "Pos", "Team", "FPT", "CR", "MIN", "Usage_Raw", "Is_Out"]].copy()

# Add Allow_FPT (DvP)
if "dvp_df" in globals() and not dvp_df.empty:
    pos_allow = dvp_df.groupby("Pos")[["Allow_FPT"]].mean().reset_index()
else:
    pos_allow = train_df.groupby("Pos")[["FPT"]].mean().rename(columns={"FPT": "Allow_FPT"}).reset_index()
train_df = train_df.merge(pos_allow, on="Pos", how="left")

# Rolling means
train_df["FPT_roll_mean"] = train_df.groupby("PlayerID")["FPT"].transform(
    lambda s: s.rolling(3, min_periods=1).mean()
)
train_df["MIN_roll_mean"] = train_df.groupby("PlayerID")["MIN"].transform(
    lambda s: s.rolling(3, min_periods=1).mean()
)

# Clean numeric fields
for col in ["FPT", "CR", "MIN", "Allow_FPT", "FPT_roll_mean", "MIN_roll_mean", "Usage_Raw"]:
    train_df[col] = pd.to_numeric(train_df[col], errors="coerce")

train_df = train_df.dropna(subset=["FPT", "CR", "MIN"]).reset_index(drop=True)

print(f"✅ Final training set: {len(train_df)} players")
print(f"✅ Features prepared: FPT_roll_mean, MIN_roll_mean, Allow_FPT, CR, Usage_Raw, Is_Out")
display(train_df[["Player", "Pos", "Team", "FPT", "FPT_roll_mean", "MIN_roll_mean", "Usage_Raw", "Is_Out"]].head(10))

✅ Downloaded 280 player rows
✅ Final training set: 280 players
✅ Features prepared: FPT_roll_mean, MIN_roll_mean, Allow_FPT, CR, Usage_Raw, Is_Out


,Player,Pos,Team,FPT,FPT_roll_mean,MIN_roll_mean,Usage_Raw,Is_Out
0,T. Dorsey,G,NaN,132.7,132.7,220.0,3.06,0
1,T. Tarpey,G,NaN,4.7,4.7,37.0,0.08,0
2,E. Okobo,G,NaN,78.0,78.0,198.0,2.29,0
3,M. Strazel,G,NaN,69.5,69.5,168.0,1.54,0
4,M. James,G,NaN,150.6,150.6,238.0,3.37,0
5,N. Nedovic,G,NaN,25.1,25.1,89.0,0.81,0
6,A. Atamna,G,NaN,7.3,7.3,72.0,0.45,0
7,G. Watson,G,NaN,93.2,93.2,189.0,2.36,0
8,M. Ngouama,G,NaN,6.5,6.5,75.0,0.83,0
9,N. De Colo,G,NaN,118.5,118.5,136.0,2.22,0


In [7]:
# =======================================================
# Cell #8 — Train + Evaluate Model
# =======================================================
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

# Define features
feature_cols = ["FPT_roll_mean", "MIN_roll_mean", "Allow_FPT", "CR", "Usage_Raw", "Is_Out"]

# Ensure all features are numeric and clean
for col in feature_cols:
    train_df[col] = pd.to_numeric(train_df[col], errors="coerce")

# Remove any rows with missing values in features or target
model_data = train_df.dropna(subset=feature_cols + ["FPT"]).copy()
print(f"📊 Model training data: {len(model_data)} samples")

if len(model_data) == 0:
    print("❌ No data available for training. Check feature calculations.")
else:
    # Prepare features and target
    X = model_data[feature_cols]
    y = model_data["FPT"]
    
    # Split data for proper evaluation (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    model = RandomForestRegressor(
        n_estimators=200, 
        max_depth=10, 
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    
    # Evaluate on both train and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    print("\n📈 MODEL PERFORMANCE:")
    print(f"   Training Set - R²: {train_r2:.3f} | MAE: {train_mae:.2f} | RMSE: {train_rmse:.2f}")
    print(f"   Test Set     - R²: {test_r2:.3f} | MAE: {test_mae:.2f} | RMSE: {test_rmse:.2f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔍 FEATURE IMPORTANCE:")
    print(feature_importance.to_string(index=False))
    
    # Make predictions on entire dataset for analysis
    model_data["Predicted_FPT"] = model.predict(X)
    model_data["Residual"] = model_data["FPT"] - model_data["Predicted_FPT"]
    
    # Show some predictions vs actual
    print("\n🔮 SAMPLE PREDICTIONS:")
    sample_results = model_data[["Player", "Pos", "Team", "FPT", "Predicted_FPT", "Residual"]].head(15)
    sample_results["Error_Pct"] = (sample_results["Residual"] / sample_results["FPT"] * 100).round(1)
    print(sample_results.to_string(index=False))
    
    # Save model predictions for later use
    predictions_df = model_data[["Player", "PlayerID", "Pos", "Team", "FPT", "Predicted_FPT", "Residual"]].copy()
    predictions_df = predictions_df.sort_values("Predicted_FPT", ascending=False)
    
    print(f"\n💾 Predictions saved for {len(predictions_df)} players")
    print("🏆 TOP 10 PREDICTED PERFORMERS:")
    print(predictions_df.head(10)[["Player", "Pos", "Team", "Predicted_FPT", "FPT"]].to_string(index=False))

📊 Model training data: 280 samples

📈 MODEL PERFORMANCE:
   Training Set - R²: 0.999 | MAE: 0.39 | RMSE: 1.24
   Test Set     - R²: 1.000 | MAE: 0.46 | RMSE: 0.76

🔍 FEATURE IMPORTANCE:
      feature  importance
FPT_roll_mean    0.997962
           CR    0.001795
MIN_roll_mean    0.000117
    Usage_Raw    0.000072
    Allow_FPT    0.000054
       Is_Out    0.000000

🔮 SAMPLE PREDICTIONS:
            Player Pos  Team   FPT  Predicted_FPT  Residual  Error_Pct
         T. Dorsey   G   NaN 132.7     132.329402  0.370598        0.3
         T. Tarpey   G   NaN   4.7       4.520964  0.179036        3.8
          E. Okobo   G   NaN  78.0      77.661444  0.338556        0.4
        M. Strazel   G   NaN  69.5      69.909584 -0.409584       -0.6
          M. James   G   NaN 150.6     153.116643 -2.516643       -1.7
        N. Nedovic   G   NaN  25.1      25.014547  0.085453        0.3
         A. Atamna   G   NaN   7.3       6.999644  0.300356        4.1
         G. Watson   G   NaN  93.2      9

In [8]:
import requests, pandas as pd, numpy as np

def _first_records(obj):
    """Find the first list[dict] inside a JSON payload."""
    if isinstance(obj, list) and (len(obj)==0 or isinstance(obj[0], dict)):
        return obj
    if isinstance(obj, dict):
        # common containers in Incrowd feeds
        for key in ["data", "games", "items", "result", "results"]:
            if key in obj:
                rec = _first_records(obj[key])
                if isinstance(rec, list) and (len(rec)==0 or isinstance(rec[0], dict)):
                    return rec
        # fallback: scan values
        for v in obj.values():
            rec = _first_records(v)
            if isinstance(rec, list) and (len(rec)==0 or isinstance(rec[0], dict)):
                return rec
    return []

def _coa(df, candidates, numeric=False, dt=False):
    """Coalesce columns by trying multiple candidates."""
    s = None
    for c in candidates:
        if c in df.columns:
            col = df[c]
            if dt:
                col = pd.to_datetime(col, errors="coerce", utc=True)
            elif numeric:
                col = pd.to_numeric(col, errors="coerce")
            s = col if s is None else s.where(s.notna(), col)
    return s

def fetch_euroleague_round_games(season_code="E2025", round_number=9, phase="RS", timeout=30):
    """
    Robust schedule fetcher for Incrowd feeds.
    Works even if keys are roundNumber/round/round.number, etc.
    """
    base = f"https://feeds.incrowdsports.com/provider/euroleague-feeds/v2/competitions/E/seasons/{season_code}/games"
    params = {"teamCode": "", "phaseTypeCode": phase, "roundNumber": round_number}
    r = requests.get(base, params=params, headers={"Accept": "application/json"}, timeout=timeout)
    r.raise_for_status()
    js = r.json()
    records = _first_records(js)
    if not records:
        print("⚠️ No games array found. Dumping top-level keys:", list(js.keys()))
        return pd.DataFrame()

    df_raw = pd.json_normalize(records, sep=".")
    # Uncomment to inspect schema if needed:
    # print("Columns:", sorted(df_raw.columns)[:40])

    round_ = _coa(df_raw, ["roundNumber","round.number","round","matchday"], numeric=True)
    date   = _coa(df_raw, ["gameDate","dateUTC","startDate","tipoff","tipoffUTC"], dt=True)
    home   = _coa(df_raw, ["homeTeam.clubName","homeTeam.teamName","homeTeamName","home.clubName","home.name"])
    away   = _coa(df_raw, ["awayTeam.clubName","awayTeam.teamName","awayTeamName","away.clubName","away.name"])
    hcode  = _coa(df_raw, ["homeTeam.teamCode","home.teamCode","home.code","homeTeamCode"])
    acode  = _coa(df_raw, ["awayTeam.teamCode","away.teamCode","away.code","awayTeamCode"])
    arena  = _coa(df_raw, ["arenaName","venueName","arena.name"])
    city   = _coa(df_raw, ["cityName","venue.city","arena.city"])

    sched = pd.DataFrame({
        "Round": round_,
        "DateUTC": date,
        "Home": home,
        "Away": away,
        "HomeCode": hcode,
        "AwayCode": acode,
        "Arena": arena,
        "City": city,
    }).dropna(subset=["Home","Away"]).sort_values(["Round","DateUTC"]).reset_index(drop=True)

    return sched

# --- Example usage ---
schedule_df = fetch_euroleague_round_games("E2025", 9)  # RS by default
print(f"Fetched {len(schedule_df)} games for Round 9")
display(schedule_df.head(10))


Fetched 10 games for Round 9


,Round,DateUTC,Home,Away,HomeCode,AwayCode,Arena,City
0,None,None,FC Barcelona,Real Madrid,BAR,MAD,None,None
1,None,None,Olympiacos Piraeus,Partizan Mozzart Bet Belgrade,OLY,PAR,None,None
2,None,None,Zalgiris Kaunas,Valencia Basket,ZAL,PAM,None,None
3,None,None,Anadolu Efes Istanbul,EA7 Emporio Armani Milan,IST,MIL,None,None
4,None,None,Paris Basketball,FC Bayern Munich,PRS,MUN,None,None
5,None,None,Maccabi Rapyd Tel Aviv,AS Monaco,TEL,MCO,None,None
6,None,None,Kosner Baskonia Vitoria-Gasteiz,Virtus Bologna,BAS,VIR,None,None
7,None,None,Dubai Basketball,Hapoel IBI Tel Aviv,DUB,HTA,None,None
8,None,None,Fenerbahce Beko Istanbul,LDLC ASVEL Villeurbanne,ULK,ASV,None,None
9,None,None,Crvena Zvezda Meridianbet Belgrade,Panathinaikos AKTOR Athens,RED,PAN,None,None


In [9]:
# --------------------------------------------
# Build `pred` DataFrame with predictions
# --------------------------------------------
pred = train_df.copy()
pred["Proj_FPT"] = y_pred
pred["Value"] = pred["Proj_FPT"] / pred["CR"]

# Normalize team names for matching
def normalize_team_name(name):
    return str(name).lower().replace("-", "").replace("basketball", "").replace("beograd", "belgrade").strip()

pred["Team_norm"] = pred["Team"].map(normalize_team_name)
dvp_df["Team_norm"] = dvp_df["Team"].map(normalize_team_name)

# Normalize schedule team names
schedule_df["Home_norm"] = schedule_df["Home"].map(normalize_team_name)
schedule_df["Away_norm"] = schedule_df["Away"].map(normalize_team_name)

# print(schedule_df.head(5))
# print(dvp_df.head(5))

# Find opponent for each team
def find_opponent(team_name):
    norm = normalize_team_name(team_name)
    row = schedule_df[(schedule_df["Home_norm"] == norm) | (schedule_df["Away_norm"] == norm)]
    if row.empty:
        return pd.Series([None, None])
    opp = row.iloc[0]["Away"] if row.iloc[0]["Home_norm"] == norm else row.iloc[0]["Home"]
    return pd.Series([opp, normalize_team_name(opp)])

pred[["OpponentName", "Opponent_norm"]] = pred["Team"].apply(find_opponent)

# Merge DvP based on opponent and player position
pred = pred.merge(
    dvp_df[["Team_norm", "Pos", "Allow_FPT"]],
    left_on=["Opponent_norm", "Pos"],
    right_on=["Team_norm", "Pos"],
    how="left",
    suffixes=("", "_opp")
)

print(pred.head(10)[["Player","Team","OpponentName","Pos","Proj_FPT","Allow_FPT"]])

# Adjust projected FPT using opponent defense
mean_dvp = dvp_df["Allow_FPT"].mean()
pred["Adj_Proj_FPT"] = pred["Proj_FPT"] * (
    pred["Allow_FPT"].fillna(mean_dvp) / mean_dvp
)

# Sort and display top players
top15 = pred.sort_values("Adj_Proj_FPT", ascending=False).head(15)
top_val = pred.sort_values("Value", ascending=False).head(5)

print("✅ Top 15 players by adjusted projected fantasy points:")
display(top15[["Player", "Pos", "Team", "OpponentName", "CR", "Proj_FPT", "Adj_Proj_FPT", "Value"]])

print("\n✅ Top 5 players by best value (Proj_FPT / CR):")
display(top_val[["Player", "Pos", "Team", "OpponentName", "CR", "Proj_FPT", "Adj_Proj_FPT", "Value"]])
""

       Player  Team OpponentName Pos    Proj_FPT  Allow_FPT
0   T. Dorsey   NaN         None   G  126.708246     13.825
1   T. Tarpey   NaN         None   G    3.338025     13.825
2    E. Okobo   NaN         None   G   91.427871     13.825
3  M. Strazel   NaN         None   G   64.244167     13.825
4    M. James   NaN         None   G  152.325948     13.825
5  N. Nedovic   NaN         None   G   29.394991     13.825
6   A. Atamna   NaN         None   G   12.055697     13.825
7   G. Watson   NaN         None   G   94.914648     13.825
8  M. Ngouama   NaN         None   G   13.989106     13.825
9  N. De Colo   NaN         None   G  112.409404     13.825
✅ Top 15 players by adjusted projected fantasy points:


,Player,Pos,Team,OpponentName,CR,Proj_FPT,Adj_Proj_FPT,Value
255,N. Milutinov,C,NaN,None,16.0,162.944860,174.780998,10.184054
13,S. Francisco,G,NaN,None,16.6,161.870343,170.811431,9.751226
60,N. Hifi,G,NaN,None,13.9,160.316185,169.171428,11.533539
200,S. Vezenkov,F,NaN,None,17.0,186.698000,162.823988,10.982235
4,M. James,G,NaN,None,16.5,152.325948,160.739841,9.231876
31,K. Nunn,G,NaN,None,15.5,149.226371,157.469055,9.627508
90,E. Bryant,G,NaN,None,16.3,143.621507,151.554600,8.811135
237,M. Kabengele,C,NaN,None,13.8,140.733534,150.956265,10.198082
156,F. Petrusev,F,NaN,None,15.4,173.002989,150.880227,11.233960
125,C. Moneke,F,NaN,None,16.1,170.149899,148.391977,10.568317



✅ Top 5 players by best value (Proj_FPT / CR):


,Player,Pos,Team,OpponentName,CR,Proj_FPT,Adj_Proj_FPT,Value
30,A. Obst,G,NaN,None,8.2,112.006719,118.193534,13.659356
142,I. Mike,F,NaN,None,9.0,117.850060,102.779980,13.094451
59,S. Herrera,G,NaN,None,5.1,63.365640,66.865711,12.424635
165,T. Luwawu-cabarrot,F,NaN,None,12.0,144.655128,126.157350,12.054594
60,N. Hifi,G,NaN,None,13.9,160.316185,169.171428,11.533539


''

In [10]:
# =======================================================
# Cell 9 - Lineup Generator (Fixed to work with our data)
# =======================================================
from itertools import combinations
from tqdm import tqdm
import random
from collections import Counter
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd

# Positional constraints for Euroleague Fantasy
MAX_BUDGET = 100.0
TOTAL_PLAYERS = 10
MIN_GUARDS = 3
MIN_FORWARDS = 3 
MIN_CENTERS = 2

print("🔄 Preparing player pool for lineup generation...")

# Create pred DataFrame from our model predictions
if 'model_data' in globals() and 'Predicted_FPT' in model_data.columns:
    pred = model_data[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT', 'Predicted_FPT']].copy()
    pred = pred.rename(columns={'Predicted_FPT': 'Adj_Proj_FPT'})
else:
    # Fallback: use the training data with FPT as projection
    pred = train_df[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT']].copy()
    pred['Adj_Proj_FPT'] = pred['FPT']  # Use historical FPT as projection

# Filter valid players (remove expensive players and those without projections)
valid_players = pred[
    (pred["CR"] <= 16) & 
    (~pred["Adj_Proj_FPT"].isna()) & 
    (pred["Adj_Proj_FPT"] > 0)
].copy().reset_index(drop=True)

print(f"✅ Player pool: {len(valid_players)} valid players")
print(f"   Guards: {len(valid_players[valid_players['Pos'] == 'G'])}")
print(f"   Forwards: {len(valid_players[valid_players['Pos'] == 'F'])}") 
print(f"   Centers: {len(valid_players[valid_players['Pos'] == 'C'])}")

# Show top players by position
print("\n🏆 TOP PLAYERS BY POSITION:")
for pos in ['G', 'F', 'C']:
    pos_players = valid_players[valid_players['Pos'] == pos].nlargest(5, 'Adj_Proj_FPT')
    print(f"\n{pos} - Top 5:")
    for _, player in pos_players.iterrows():
        print(f"   {player['Player']} | {player['Team']} | CR: {player['CR']} | Proj: {player['Adj_Proj_FPT']:.1f}")

# =======================================================
# METHOD 1: Brute Force with Progress Bar (Limited)
# =======================================================
print("\n" + "="*50)
print("METHOD 1: Brute Force Search")
print("="*50)

# Store top teams
best_teams = []
MAX_TRIALS = 50000  # Reduced for reasonable runtime

# Create position indices for faster filtering
guard_indices = valid_players[valid_players["Pos"] == "G"].index.tolist()
forward_indices = valid_players[valid_players["Pos"] == "F"].index.tolist()  
center_indices = valid_players[valid_players["Pos"] == "C"].index.tolist()

print(f"Searching up to {MAX_TRIALS:,} random combinations...")

with tqdm(total=MAX_TRIALS) as pbar:
    for trial in range(MAX_TRIALS):
        # Generate random team
        guards = random.sample(guard_indices, MIN_GUARDS) if len(guard_indices) >= MIN_GUARDS else []
        forwards = random.sample(forward_indices, MIN_FORWARDS) if len(forward_indices) >= MIN_FORWARDS else []
        centers = random.sample(center_indices, MIN_CENTERS) if len(center_indices) >= MIN_CENTERS else []
        
        # Fill remaining spots randomly
        remaining = TOTAL_PLAYERS - (MIN_GUARDS + MIN_FORWARDS + MIN_CENTERS)
        all_indices = guard_indices + forward_indices + center_indices
        remaining_indices = random.sample(all_indices, remaining)
        
        team_indices = guards + forwards + centers + remaining_indices
        team_indices = list(set(team_indices))  # Remove duplicates
        
        if len(team_indices) != TOTAL_PLAYERS:
            pbar.update(1)
            continue
            
        team = valid_players.loc[team_indices]
        budget = team["CR"].sum()
        pos_counts = team["Pos"].value_counts().to_dict()
        
        # Check constraints
        if (budget <= MAX_BUDGET and 
            pos_counts.get("G", 0) >= MIN_GUARDS and
            pos_counts.get("F", 0) >= MIN_FORWARDS and 
            pos_counts.get("C", 0) >= MIN_CENTERS):
            
            total_score = team["Adj_Proj_FPT"].sum()
            best_teams.append((total_score, budget, team.copy()))
            
        pbar.update(1)

# Show best teams
if best_teams:
    best_teams = sorted(best_teams, reverse=True, key=lambda x: x[0])[:5]
    print(f"\n🎯 Top {len(best_teams)} Teams Found (Brute Force):")
    
    for i, (score, budget, team) in enumerate(best_teams, 1):
        pos_counts = team["Pos"].value_counts().to_dict()
        print(f"\n💡 Team #{i} | Score: {score:.2f} | Budget: {budget:.1f} | Positions: {pos_counts}")
        display(team[["Player", "Pos", "Team", "CR", "Adj_Proj_FPT"]].sort_values("Pos"))
else:
    print("❌ No valid teams found with brute force method")

# =======================================================
# METHOD 2: Multi-Shot Greedy with Randomization
# =======================================================
print("\n" + "="*50)
print("METHOD 2: Greedy Randomized Search")
print("="*50)

POS_MIN = {"G": MIN_GUARDS, "F": MIN_FORWARDS, "C": MIN_CENTERS}
POS_MAX = {"G": 5, "F": 5, "C": 4}  # More flexible maxes
NUM_TRIALS = 10000

best_greedy_team = None
best_greedy_score = 0
best_greedy_budget = 0
best_greedy_pos_counts = {}

print(f"Running {NUM_TRIALS} greedy randomized trials...")

for trial in tqdm(range(NUM_TRIALS)):
    # Shuffle players differently each time
    shuffled = valid_players.copy().sample(frac=1, random_state=trial).reset_index(drop=True)
    team = []
    budget = 0
    pos_counts = {"G": 0, "F": 0, "C": 0}
    
    for _, player in shuffled.iterrows():
        pos = player["Pos"]
        cost = player["CR"]
        
        # Skip if over budget or position max
        if (cost + budget > MAX_BUDGET or 
            pos_counts[pos] >= POS_MAX[pos]):
            continue
            
        team.append(player)
        budget += cost
        pos_counts[pos] += 1
        
        # Stop when we have full team
        if len(team) == TOTAL_PLAYERS:
            break
    
    # Check if team meets minimum position requirements
    if (len(team) == TOTAL_PLAYERS and 
        all(pos_counts[p] >= POS_MIN[p] for p in POS_MIN)):
        
        score = sum(p["Adj_Proj_FPT"] for p in team)
        if score > best_greedy_score:
            best_greedy_team = team
            best_greedy_score = score
            best_greedy_budget = budget
            best_greedy_pos_counts = pos_counts.copy()

# Show greedy result
if best_greedy_team:
    best_greedy_df = pd.DataFrame(best_greedy_team)
    best_greedy_df = best_greedy_df.sort_values("Pos")
    
    print(f"✅ Best Greedy Team | Score: {best_greedy_score:.2f} | Budget: {best_greedy_budget:.1f} | Positions: {best_greedy_pos_counts}")
    display(best_greedy_df[["Player", "Pos", "Team", "CR", "Adj_Proj_FPT"]])
else:
    print("❌ No valid team found with greedy method")

# =======================================================
# METHOD 3: Genetic Algorithm
# =======================================================
print("\n" + "="*50)
print("METHOD 3: Genetic Algorithm")
print("="*50)

# Genetic Algorithm parameters
NUM_GENERATIONS = 50
POP_SIZE = 100
MUTATION_RATE = 0.3
ELITE_SIZE = 10

print(f"Running genetic algorithm for {NUM_GENERATIONS} generations...")

# Create positional pools
guards = valid_players[valid_players["Pos"] == "G"].index.tolist()
forwards = valid_players[valid_players["Pos"] == "F"].index.tolist()
centers = valid_players[valid_players["Pos"] == "C"].index.tolist()
player_pool = valid_players.index.tolist()

def evaluate_team(indices):
    """Evaluate a team and return its score (0 if invalid)"""
    team = valid_players.loc[indices]
    pos_counts = Counter(team["Pos"])
    budget = team["CR"].sum()
    score = team["Adj_Proj_FPT"].sum()

    # Check constraints
    if (budget > MAX_BUDGET or
        pos_counts.get("G", 0) < MIN_GUARDS or
        pos_counts.get("F", 0) < MIN_FORWARDS or
        pos_counts.get("C", 0) < MIN_CENTERS):
        return 0
        
    return score

def generate_random_team():
    """Generate a random valid team"""
    for _ in range(1000):  # Try up to 1000 times
        # Start with minimum required positions
        guards_selected = random.sample(guards, MIN_GUARDS) if len(guards) >= MIN_GUARDS else []
        forwards_selected = random.sample(forwards, MIN_FORWARDS) if len(forwards) >= MIN_FORWARDS else []
        centers_selected = random.sample(centers, MIN_CENTERS) if len(centers) >= MIN_CENTERS else []
        
        selected = guards_selected + forwards_selected + centers_selected
        
        # Fill remaining spots
        remaining = TOTAL_PLAYERS - len(selected)
        if remaining > 0:
            additional = random.sample(player_pool, remaining)
            selected.extend(additional)
        
        selected = list(set(selected))  # Remove duplicates
        
        if len(selected) == TOTAL_PLAYERS and evaluate_team(selected) > 0:
            return selected
    
    # Fallback: return any team and let evolution fix it
    return random.sample(player_pool, TOTAL_PLAYERS)

# Initialize population
population = [generate_random_team() for _ in range(POP_SIZE)]

# Evolution loop
for generation in range(NUM_GENERATIONS):
    # Evaluate and sort population
    scored_population = [(evaluate_team(team), team) for team in population]
    scored_population.sort(reverse=True, key=lambda x: x[0])
    
    # Keep elites
    new_population = [team for score, team in scored_population[:ELITE_SIZE]]
    
    # Breed new individuals
    while len(new_population) < POP_SIZE:
        if random.random() < 0.7:  # Crossover
            parent1 = random.choice(scored_population[:ELITE_SIZE*2])[1]
            parent2 = random.choice(scored_population[:ELITE_SIZE*2])[1]
            
            # Single-point crossover
            crossover_point = random.randint(1, TOTAL_PLAYERS-1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            child = list(set(child))  # Remove duplicates
            
            # Fill or trim to correct size
            if len(child) > TOTAL_PLAYERS:
                child = random.sample(child, TOTAL_PLAYERS)
            elif len(child) < TOTAL_PLAYERS:
                needed = TOTAL_PLAYERS - len(child)
                additional = random.sample(player_pool, needed)
                child.extend(additional)
        else:  # Mutation
            parent = random.choice(scored_population[:ELITE_SIZE*2])[1]
            child = parent.copy()
            
            # Mutate some players
            for i in range(TOTAL_PLAYERS):
                if random.random() < MUTATION_RATE:
                    child[i] = random.choice(player_pool)
            child = list(set(child))  # Remove duplicates
            
            # Ensure correct size
            if len(child) != TOTAL_PLAYERS:
                child = generate_random_team()
                
        new_population.append(child)
    
    population = new_population

# Get final best teams
final_scored = [(evaluate_team(team), team) for team in population]
final_scored.sort(reverse=True, key=lambda x: x[0])

# Display top genetic algorithm teams
top_genetic_teams = []
for score, team_idx in final_scored:
    if score > 0 and team_idx not in [t[1] for t in top_genetic_teams]:
        top_genetic_teams.append((score, team_idx))
    if len(top_genetic_teams) >= 3:
        break

if top_genetic_teams:
    print(f"\n🎯 Top {len(top_genetic_teams)} Genetic Algorithm Teams:")
    for i, (score, team_idx) in enumerate(top_genetic_teams, 1):
        team_df = valid_players.loc[team_idx].sort_values("Pos")
        budget = team_df["CR"].sum()
        pos_counts = team_df["Pos"].value_counts().to_dict()
        print(f"\n💥 Team #{i} | Score: {score:.2f} | Budget: {budget:.1f} | Positions: {pos_counts}")
        display(team_df[["Player", "Pos", "Team", "CR", "Adj_Proj_FPT"]])
else:
    print("❌ No valid teams found with genetic algorithm")

# =======================================================
# METHOD 4: Integer Linear Programming (Optimal)
# =======================================================
print("\n" + "="*50)
print("METHOD 4: Integer Linear Programming (Optimal)")
print("="*50)

try:
    # Position constraints
    POS_MIN = {"G": MIN_GUARDS, "F": MIN_FORWARDS, "C": MIN_CENTERS}
    POS_MAX = {"G": 6, "F": 6, "C": 4}  # More flexible maximums
    TOP_N = 3

    print("Solving ILP problem...")

    # Create the optimization problem
    prob = LpProblem("Fantasy_Lineup_Optimization", LpMaximize)

    # Decision variables: 1 if player i is selected, 0 otherwise
    player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(valid_players))]

    # Objective: maximize total projected points
    prob += lpSum(valid_players.iloc[i]["Adj_Proj_FPT"] * player_vars[i] for i in range(len(valid_players)))

    # Constraints
    # Total players constraint
    prob += lpSum(player_vars) == TOTAL_PLAYERS

    # Budget constraint
    prob += lpSum(valid_players.iloc[i]["CR"] * player_vars[i] for i in range(len(valid_players))) <= MAX_BUDGET

    # Position constraints
    for pos in POS_MIN:
        pos_players = [i for i in range(len(valid_players)) if valid_players.iloc[i]["Pos"] == pos]
        prob += lpSum(player_vars[i] for i in pos_players) >= POS_MIN[pos]
        prob += lpSum(player_vars[i] for i in pos_players) <= POS_MAX[pos]

    # Solve the problem
    prob.solve()

    if LpStatus[prob.status] == "Optimal":
        # Get selected players
        selected_indices = [i for i in range(len(valid_players)) if player_vars[i].value() == 1]
        optimal_team = valid_players.iloc[selected_indices].copy()
        
        total_score = optimal_team["Adj_Proj_FPT"].sum()
        total_budget = optimal_team["CR"].sum()
        pos_counts = optimal_team["Pos"].value_counts().to_dict()
        
        print(f"✅ OPTIMAL TEAM FOUND!")
        print(f"   Score: {total_score:.2f} | Budget: {total_budget:.1f} | Positions: {pos_counts}")
        print(f"   Solution status: {LpStatus[prob.status]}")
        
        display(optimal_team[["Player", "Pos", "Team", "CR", "Adj_Proj_FPT"]].sort_values("Pos"))
        
    else:
        print(f"❌ No optimal solution found. Status: {LpStatus[prob.status]}")
        
except Exception as e:
    print(f"❌ ILP solver failed: {e}")
    print("   This is often due to PuLP/CBC solver not being installed properly.")

print("\n" + "="*50)
print("SUMMARY: All lineup generation methods completed!")
print("="*50)

🔄 Preparing player pool for lineup generation...
✅ Player pool: 256 valid players
   Guards: 105
   Forwards: 101
   Centers: 50

🏆 TOP PLAYERS BY POSITION:

G - Top 5:
   N. Hifi | nan | CR: 13.9 | Proj: 174.0
   K. Nunn | nan | CR: 15.5 | Proj: 147.8
   W. Baldwin Iv | nan | CR: 12.4 | Proj: 137.0
   T. Dorsey | nan | CR: 11.5 | Proj: 132.3
   O. Moore | nan | CR: 11.2 | Proj: 132.1

F - Top 5:
   F. Petrusev | nan | CR: 15.4 | Proj: 180.0
   T. Shengelia | nan | CR: 15.5 | Proj: 156.0
   T. Luwawu-cabarrot | nan | CR: 12.0 | Proj: 153.9
   J. Nwora | nan | CR: 12.7 | Proj: 151.5
   T. Lyles | nan | CR: 14.0 | Proj: 132.0

C - Top 5:
   N. Milutinov | nan | CR: 16.0 | Proj: 181.1
   M. Wright | nan | CR: 12.5 | Proj: 149.8
   M. Kabengele | nan | CR: 13.8 | Proj: 147.0
   D. Theis | nan | CR: 13.3 | Proj: 146.5
   T. Jones | nan | CR: 15.2 | Proj: 137.7

METHOD 1: Brute Force Search
Searching up to 50,000 random combinations...


100%|██████████| 50000/50000 [01:33<00:00, 533.49it/s]



🎯 Top 5 Teams Found (Brute Force):

💡 Team #1 | Score: 965.89 | Budget: 99.2 | Positions: {'G': 4, 'F': 3, 'C': 3}


,Player,Pos,Team,CR,Adj_Proj_FPT
235,N. Milutinov,C,NaN,16.0,181.052550
208,D. Motiejunas,C,NaN,7.5,73.984596
209,M. Wright,C,NaN,12.5,149.816635
129,I. Mike,F,NaN,9.0,122.725605
133,J. Hernangomez,F,NaN,13.1,114.592740
198,G. Grinvalds,F,NaN,4.0,0.000872
64,O. Moore,G,NaN,11.2,132.149108
37,J. Dibartolomeo,G,NaN,5.6,17.051246
71,D. Hall,G,NaN,9.7,104.341949
84,C. Jones,G,NaN,10.6,70.171653



💡 Team #2 | Score: 964.10 | Budget: 95.1 | Positions: {'F': 4, 'C': 3, 'G': 3}


...


💥 Team #3 | Score: 1090.63 | Budget: 92.7 | Positions: {'F': 5, 'G': 3, 'C': 2}


,Player,Pos,Team,CR,Adj_Proj_FPT
204,M. Ndiaye,C,NaN,7.5,87.246952
237,D. Oturu,C,NaN,12.0,137.212019
129,I. Mike,F,NaN,9.0,122.725605
131,O. Da Silva,F,NaN,7.2,81.522635
165,E. Osmani,F,NaN,9.4,110.217146
107,M. Ajinca,F,NaN,4.0,14.315091
190,J. Nwora,F,NaN,12.7,151.498156
0,T. Dorsey,G,NaN,11.5,132.329402
64,O. Moore,G,NaN,11.2,132.149108
28,A. Obst,G,NaN,8.2,121.416689



METHOD 4: Integer Linear Programming (Optimal)
Solving ILP problem...
✅ OPTIMAL TEAM FOUND!
   Score: 1249.78 | Budget: 99.8 | Positions: {'G': 4, 'F': 4, 'C': 2}
   Solution status: Optimal


,Player,Pos,Team,CR,Adj_Proj_FPT
204,M. Ndiaye,C,NaN,7.5,87.246952
209,M. Wright,C,NaN,12.5,149.816635
129,I. Mike,F,NaN,9.0,122.725605
151,T. Luwawu-cabarrot,F,NaN,12.0,153.930785
169,N. Reuvers,F,NaN,7.7,87.168168
190,J. Nwora,F,NaN,12.7,151.498156
28,A. Obst,G,NaN,8.2,121.416689
55,S. Herrera,G,NaN,5.1,69.877093
56,N. Hifi,G,NaN,13.9,173.953271
64,O. Moore,G,NaN,11.2,132.149108



SUMMARY: All lineup generation methods completed!


In [11]:
# =======================================================
# Cell 9 - OPTIMIZED Lineup Generator (No Starter/Momentum Boosts)
# =======================================================
from itertools import combinations
from tqdm import tqdm
import random
from collections import Counter
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
import numpy as np
from datetime import datetime
import os

print("🚀 OPTIMIZED Lineup Generation with Model Integration")

# =======================================================
# STEP 1: Create Enhanced Projections WITHOUT Starter/Momentum Boosts
# =======================================================

def create_enhanced_projections_no_boosts():
    """Create projections WITHOUT starter and momentum boosts"""
    
    # Start with model predictions
    if 'model_data' in globals() and 'Predicted_FPT' in model_data.columns:
        projections = model_data[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT', 'Predicted_FPT']].copy()
        projections = projections.rename(columns={'Predicted_FPT': 'Base_Projection'})
    else:
        # Fallback to training data
        projections = train_df[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT']].copy()
        projections['Base_Projection'] = projections['FPT']
    
    # Add model features for context
    feature_cols = ['FPT_roll_mean', 'MIN_roll_mean', 'Allow_FPT', 'Usage_Raw', 'Is_Out']
    for col in feature_cols:
        if col in train_df.columns:
            projections = projections.merge(
                train_df[['PlayerID', col]].drop_duplicates(), 
                on='PlayerID', how='left'
            )
    
    # =======================================================
    # REMOVED: Starter Status Boost (keep for info only)
    # =======================================================
    if 'df_lineups' in globals() and not df_lineups.empty:
        print("📋 Recording starter status (NO BOOST APPLIED)...")
        starter_players = []
        for _, row in df_lineups.iterrows():
            for i in range(1, 6):
                starter_col = f'starter_{i}'
                if starter_col in row and pd.notna(row[starter_col]):
                    starter_players.append(row[starter_col].strip().lower())
        
        starter_players = set(starter_players)
        projections['Is_Starter'] = projections['Player'].str.lower().isin(starter_players).astype(int)
    else:
        projections['Is_Starter'] = 0
    
    # =======================================================
    # ENHANCEMENT: Incorporate Injury Status from MNP
    # =======================================================
    if 'mnp_df' in globals() and not mnp_df.empty:
        print("🏥 Incorporating injury status from MNP data...")
        injured_players = set(mnp_df['player'].str.lower().dropna())
        projections['Is_Injured'] = projections['Player'].str.lower().isin(injured_players).astype(int)
        
        # Penalize injured players
        projections['Injury_Penalty'] = projections['Is_Injured'] * projections['Base_Projection'] * 0.5
    else:
        projections['Is_Injured'] = 0
        projections['Injury_Penalty'] = 0
    
    # =======================================================
    # REMOVED: Momentum Boost (keep ratio for info only)
    # =======================================================
    if 'FPT_roll_mean' in projections.columns:
        projections['Momentum_Ratio'] = projections['FPT_roll_mean'] / projections['Base_Projection'].replace(0, 1)
    
    # =======================================================
    # ENHANCEMENT: Usage Rate Integration
    # =======================================================
    if 'Usage_Raw' in projections.columns:
        usage_mean = projections['Usage_Raw'].mean()
        projections['Usage_Boost'] = (
            (projections['Usage_Raw'] - usage_mean) / usage_mean * projections['Base_Projection'] * 0.2
        )
    else:
        projections['Usage_Boost'] = 0
    
    # =======================================================
    # ENHANCEMENT: Defense vs Position (DvP) Adjustment
    # =======================================================
    if 'Allow_FPT' in projections.columns:
        dvp_mean = projections['Allow_FPT'].mean()
        projections['DvP_Boost'] = (
            (projections['Allow_FPT'] - dvp_mean) / dvp_mean * projections['Base_Projection'] * 0.15
        )
    else:
        projections['DvP_Boost'] = 0
    
    # =======================================================
    # FINAL ENHANCED PROJECTION (WITHOUT STARTER & MOMENTUM BOOSTS)
    # =======================================================
    projections['Enhanced_Projection'] = (
        projections['Base_Projection'] + 
        projections['Usage_Boost'] +
        projections['DvP_Boost'] -
        projections['Injury_Penalty']
    )
    
    # Ensure projections are reasonable
    projections['Enhanced_Projection'] = projections['Enhanced_Projection'].clip(lower=0)
    
    # Calculate Value Ratio
    projections['Value_Ratio'] = projections['Enhanced_Projection'] / projections['CR']
    
    print("✅ Projections created WITHOUT starter/momentum boosts")
    
    return projections

# Create enhanced projections WITHOUT starter and momentum boosts
enhanced_projections = create_enhanced_projections_no_boosts()

print(f"✅ Created enhanced projections for {len(enhanced_projections)} players")

# =======================================================
# STEP 2: Export ALL Player Projections to CSV in Euroleague Folder
# =======================================================

def export_all_projections_to_csv(projections_df, folder="Euroleague"):
    """Export comprehensive player projections to CSV in specified folder"""
    
    # Create Euroleague folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"📁 Created folder: {folder}")
    
    # Select and order columns for the export
    export_columns = [
        'Player', 'Pos', 'Team', 'CR', 
        'Base_Projection', 'Enhanced_Projection', 'Value_Ratio',
        'Is_Starter', 'Is_Injured'
    ]
    
    # Add remaining enhancement columns
    enhancement_cols = ['Usage_Boost', 'DvP_Boost', 'Injury_Penalty']
    for col in enhancement_cols:
        if col in projections_df.columns:
            export_columns.append(col)
    
    # Add model features if they exist
    feature_cols = ['FPT_roll_mean', 'MIN_roll_mean', 'Usage_Raw', 'Allow_FPT']
    for col in feature_cols:
        if col in projections_df.columns:
            export_columns.append(col)
    
    # Create export DataFrame with selected columns
    export_df = projections_df[export_columns].copy()
    
    # Sort by Enhanced Projection (most valuable players first)
    export_df = export_df.sort_values('Enhanced_Projection', ascending=False)
    
    # Format numeric columns for better readability
    numeric_cols = ['Base_Projection', 'Enhanced_Projection', 'Value_Ratio'] + enhancement_cols
    for col in numeric_cols:
        if col in export_df.columns:
            if col == 'Value_Ratio':
                export_df[col] = export_df[col].round(3)
            else:
                export_df[col] = export_df[col].round(2)
    
    # Create timestamp for filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"euroleague_player_projections_{timestamp}.csv"
    filepath = os.path.join(folder, filename)
    
    # Export to CSV
    export_df.to_csv(filepath, index=False, encoding='utf-8')
    
    print(f"💾 Exported {len(export_df)} player projections to: {filepath}")
    
    # Show summary statistics
    print(f"\n📊 PROJECTIONS SUMMARY:")
    print(f"   Highest Projection: {export_df['Enhanced_Projection'].max():.2f}")
    print(f"   Average Projection: {export_df['Enhanced_Projection'].mean():.2f}")
    print(f"   Total Starters: {export_df['Is_Starter'].sum()}")
    print(f"   Total Injured: {export_df['Is_Injured'].sum()}")
    
    return export_df, filepath

# Export all projections to CSV in Euroleague folder
all_projections_df, csv_filepath = export_all_projections_to_csv(enhanced_projections, "Euroleague")

# Display top 20 players for preview
print(f"\n🏆 TOP 20 PLAYERS BY ENHANCED PROJECTION:")
preview_cols = ['Player', 'Pos', 'Team', 'CR', 'Base_Projection', 'Enhanced_Projection', 'Value_Ratio', 'Is_Starter']
preview_cols = [col for col in preview_cols if col in all_projections_df.columns]
display(all_projections_df[preview_cols].head(20))

# =======================================================
# STEP 3: Smart Player Pool Filtering for 100 Credit Budget
# =======================================================

# Euroleague Fantasy constraints with 100 credit budget
MAX_BUDGET = 100.0
TOTAL_PLAYERS = 10
POSITION_RULES = {
    'G': {'min': 3, 'max': 5, 'ideal': 4},
    'F': {'min': 3, 'max': 5, 'ideal': 4}, 
    'C': {'min': 2, 'max': 3, 'ideal': 2}
}

def create_optimized_player_pool(projections_df, max_credit=100):
    """Create optimized player pool for 100 credit budget"""
    
    avg_player_cost = max_credit / TOTAL_PLAYERS
    
    # Remove expensive players that would break budget
    filtered = projections_df[
        (projections_df['Is_Injured'] == 0) &  # Remove injured players
        (projections_df['CR'] <= 15) &         # Lower max cost for 100 credit budget
        (projections_df['CR'] >= 4) &          # Avoid very cheap low-projection players
        (projections_df['Enhanced_Projection'] > 5)  # Minimum projection
    ].copy()
    
    # Position-specific value thresholds
    pos_value_thresholds = {}
    for pos in ['G', 'F', 'C']:
        pos_players = filtered[filtered['Pos'] == pos]
        if len(pos_players) > 0:
            threshold = pos_players['Value_Ratio'].quantile(0.4)  # Keep top 60% by value
            pos_value_thresholds[pos] = threshold
    
    # Apply position-specific filtering
    keep_mask = pd.Series(False, index=filtered.index)
    for pos, threshold in pos_value_thresholds.items():
        pos_mask = (filtered['Pos'] == pos) & (filtered['Value_Ratio'] >= threshold)
        keep_mask = keep_mask | pos_mask
    
    filtered = filtered[keep_mask].copy()
    
    print(f"💰 BUDGET STRATEGY: {max_credit} credits")
    print(f"   Average target cost per player: {avg_player_cost:.1f} credits")
    
    return filtered

optimized_pool = create_optimized_player_pool(enhanced_projections, MAX_BUDGET)

print(f"\n🎯 OPTIMIZED PLAYER POOL FOR {MAX_BUDGET} CREDIT BUDGET:")
print(f"   Total players: {len(optimized_pool)}")
print(f"   Guards: {len(optimized_pool[optimized_pool['Pos'] == 'G'])}")
print(f"   Forwards: {len(optimized_pool[optimized_pool['Pos'] == 'F'])}")
print(f"   Centers: {len(optimized_pool[optimized_pool['Pos'] == 'C'])}")
print(f"   Average cost: {optimized_pool['CR'].mean():.1f} credits")

# =======================================================
# STEP 4: Improved Lineup Optimization for 100 Credits
# =======================================================

print(f"\n{'='*60}")
print(f"OPTIMIZED INTEGER LINEAR PROGRAMMING SOLUTION - {MAX_BUDGET} CREDITS")
print(f"{'='*60}")

try:
    # Create the optimization problem
    prob = LpProblem("Euroleague_Fantasy_Optimization", LpMaximize)
    
    # Decision variables
    player_vars = LpVariable.dicts("Player", optimized_pool.index, cat=LpBinary)
    
    # Objective: Maximize enhanced projections
    prob += lpSum(
        optimized_pool.loc[i, "Enhanced_Projection"] * player_vars[i] 
        for i in optimized_pool.index
    )
    
    # Total players constraint
    prob += lpSum(player_vars[i] for i in optimized_pool.index) == TOTAL_PLAYERS
    
    # Budget constraint
    prob += lpSum(
        optimized_pool.loc[i, "CR"] * player_vars[i] 
        for i in optimized_pool.index
    ) <= MAX_BUDGET
    
    # Position constraints
    for pos, rules in POSITION_RULES.items():
        pos_players = optimized_pool[optimized_pool["Pos"] == pos].index
        prob += lpSum(player_vars[i] for i in pos_players) >= rules['min']
        prob += lpSum(player_vars[i] for i in pos_players) <= rules['max']
    
    # Solve
    prob.solve()
    
    if LpStatus[prob.status] == "Optimal":
        # Get selected players
        selected_indices = [i for i in optimized_pool.index if player_vars[i].value() == 1]
        optimal_team = optimized_pool.loc[selected_indices].copy()
        
        total_score = optimal_team["Enhanced_Projection"].sum()
        total_budget = optimal_team["CR"].sum()
        pos_counts = optimal_team["Pos"].value_counts().to_dict()
        
        print("🎉 OPTIMAL LINEUP FOUND!")
        print(f"   Enhanced Score: {total_score:.2f}")
        print(f"   Budget Used: {total_budget:.1f}/{MAX_BUDGET}")
        print(f"   Budget Remaining: {MAX_BUDGET - total_budget:.1f}")
        print(f"   Position Distribution: {pos_counts}")
        
        # Display optimal team
        optimal_team_sorted = optimal_team.sort_values(["Pos", "Enhanced_Projection"], ascending=[True, False])
        display_cols = ["Player", "Pos", "Team", "CR", "Base_Projection", "Enhanced_Projection", "Is_Starter"]
        display_cols = [col for col in display_cols if col in optimal_team_sorted.columns]
        
        print("\n🏀 OPTIMAL LINEUP:")
        display(optimal_team_sorted[display_cols])
        
        # Export optimal lineup to CSV
        lineup_filename = f"optimal_lineup_{MAX_BUDGET}credits_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        lineup_filepath = os.path.join("Euroleague", lineup_filename)
        optimal_team_sorted[display_cols].to_csv(lineup_filepath, index=False)
        print(f"💾 Optimal lineup saved to: {lineup_filepath}")
        
    else:
        print(f"❌ No optimal solution found. Status: {LpStatus[prob.status]}")
        
except Exception as e:
    print(f"❌ ILP solver failed: {e}")

# =======================================================
# STEP 5: Export Position Rankings and Show Folder Contents
# =======================================================

def export_position_rankings(projections_df, folder="Euroleague"):
    """Export separate CSV files for each position ranking"""
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    for position in ['G', 'F', 'C']:
        pos_players = projections_df[projections_df['Pos'] == position].copy()
        pos_players = pos_players.sort_values('Enhanced_Projection', ascending=False)
        
        pos_columns = ['Player', 'Team', 'CR', 'Base_Projection', 'Enhanced_Projection', 'Value_Ratio', 'Is_Starter']
        pos_columns = [col for col in pos_columns if col in pos_players.columns]
        
        filename = f"{position}_rankings_{timestamp}.csv"
        filepath = os.path.join(folder, filename)
        pos_players[pos_columns].to_csv(filepath, index=False)
        
        print(f"💾 {position} rankings exported: {filepath}")

# Export position rankings
export_position_rankings(all_projections_df, "Euroleague")

# Show folder contents
print(f"\n{'='*60}")
print("EXPORT SUMMARY")
print(f"{'='*60}")

current_dir = os.getcwd()
euroleague_folder = os.path.join(current_dir, "Euroleague")

print(f"📁 Euroleague export folder: {euroleague_folder}")
print(f"💰 Budget constraint: {MAX_BUDGET} credits")

if os.path.exists("Euroleague"):
    print(f"\n📋 Files in Euroleague folder:")
    euroleague_files = os.listdir("Euroleague")
    for file in euroleague_files:
        if file.endswith('.csv'):
            file_path = os.path.join("Euroleague", file)
            file_size = os.path.getsize(file_path)
            print(f"   📄 {file} ({file_size} bytes)")

print(f"\n✅ ALL EXPORTS COMPLETE!")
print(f"✨ Projections created WITHOUT starter/momentum boosts")

🚀 OPTIMIZED Lineup Generation with Model Integration
📋 Recording starter status (NO BOOST APPLIED)...
🏥 Incorporating injury status from MNP data...
✅ Projections created WITHOUT starter/momentum boosts
✅ Created enhanced projections for 280 players
💾 Exported 280 player projections to: Euroleague\euroleague_player_projections_20251126_170113.csv

📊 PROJECTIONS SUMMARY:
   Highest Projection: 265.93
   Average Projection: 59.44
   Total Starters: 45
   Total Injured: 18

🏆 TOP 20 PLAYERS BY ENHANCED PROJECTION:


,Player,Pos,Team,CR,Base_Projection,Enhanced_Projection,Value_Ratio,Is_Starter
60,N. Hifi,G,NaN,13.9,173.95,265.93,19.131,0
200,S. Vezenkov,F,NaN,17.0,181.43,239.72,14.101,1
156,F. Petrusev,F,NaN,15.4,180.04,226.47,14.706,1
125,C. Moneke,F,NaN,16.1,179.71,225.11,13.982,0
13,S. Francisco,G,NaN,16.6,162.75,219.56,13.227,1
4,M. James,G,NaN,16.5,153.12,214.93,13.026,0
31,K. Nunn,G,NaN,15.5,147.79,212.66,13.720,0
165,T. Luwawu-cabarrot,F,NaN,12.0,153.93,208.00,17.333,0
255,N. Milutinov,C,NaN,16.0,181.05,205.19,12.824,1
170,T. Shengelia,F,NaN,15.5,156.01,190.20,12.271,1


💰 BUDGET STRATEGY: 100.0 credits
   Average target cost per player: 10.0 credits

🎯 OPTIMIZED PLAYER POOL FOR 100.0 CREDIT BUDGET:
   Total players: 126
   Guards: 52
   Forwards: 52
   Centers: 22
   Average cost: 9.7 credits

OPTIMIZED INTEGER LINEAR PROGRAMMING SOLUTION - 100.0 CREDITS
🎉 OPTIMAL LINEUP FOUND!
   Enhanced Score: 1573.81
   Budget Used: 99.8/100.0
   Budget Remaining: 0.2
   Position Distribution: {'G': 5, 'F': 3, 'C': 2}

🏀 OPTIMAL LINEUP:


,Player,Pos,Team,CR,Base_Projection,Enhanced_Projection,Is_Starter
257,D. Oturu,C,NaN,12.0,137.212019,160.579090,0
222,M. Ndiaye,C,NaN,7.5,87.246952,91.041027,0
165,T. Luwawu-cabarrot,F,NaN,12.0,153.930785,207.995540,0
142,I. Mike,F,NaN,9.0,122.725605,145.727864,0
179,E. Osmani,F,NaN,9.4,110.217146,130.874965,0
60,N. Hifi,G,NaN,13.9,173.953271,265.926092,0
0,T. Dorsey,G,NaN,11.5,132.329402,178.521697,0
69,O. Moore,G,NaN,11.2,132.149108,168.037379,0
30,A. Obst,G,NaN,8.2,121.416689,147.333291,0
59,S. Herrera,G,NaN,5.1,69.877093,77.777298,1


💾 Optimal lineup saved to: Euroleague\optimal_lineup_100.0credits_20251126_170113.csv
💾 G rankings exported: Euroleague\G_rankings_20251126_170113.csv
💾 F rankings exported: Euroleague\F_rankings_20251126_170113.csv
💾 C rankings exported: Euroleague\C_rankings_20251126_170113.csv

EXPORT SUMMARY
📁 Euroleague export folder: c:\Users\minas\Desktop\NBA\NBA_value_bets_exporter\Euroleague\Euroleague
💰 Budget constraint: 100.0 credits

📋 Files in Euroleague folder:
   📄 C_rankings_20251105_214538.csv (2014 bytes)
   📄 C_rankings_20251108_222052.csv (2009 bytes)
   📄 C_rankings_20251111_184606.csv (2009 bytes)
   📄 C_rankings_20251114_010038.csv (2012 bytes)
   📄 C_rankings_20251115_210826.csv (2016 bytes)
   📄 C_rankings_20251119_234740.csv (2018 bytes)
   📄 C_rankings_20251121_191234.csv (2013 bytes)
   📄 C_rankings_20251126_170113.csv (2020 bytes)
   📄 euroleague_complete_team_20251108_222109.csv (505 bytes)
   📄 euroleague_complete_team_20251111_184711.csv (508 bytes)
   📄 euroleague_comp

In [12]:
# =======================================================
# Cell 9 - OPTIMIZED Lineup Generator with Model Integration
# =======================================================
from itertools import combinations
from tqdm import tqdm
import random
from collections import Counter
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus
import pandas as pd
import numpy as np

print("🚀 OPTIMIZED Lineup Generation with Model Integration")

# =======================================================
# STEP 1: Create Enhanced Projections Using Model + Context
# =======================================================

def create_enhanced_projections():
    """Create projections that fully utilize our model and features"""
    
    # Start with model predictions
    if 'model_data' in globals() and 'Predicted_FPT' in model_data.columns:
        projections = model_data[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT', 'Predicted_FPT']].copy()
        projections = projections.rename(columns={'Predicted_FPT': 'Base_Projection'})
    else:
        # Fallback to training data
        projections = train_df[['Player', 'PlayerID', 'Pos', 'Team', 'CR', 'FPT']].copy()
        projections['Base_Projection'] = projections['FPT']
    
    # Add model features for context
    feature_cols = ['FPT_roll_mean', 'MIN_roll_mean', 'Allow_FPT', 'Usage_Raw', 'Is_Out']
    for col in feature_cols:
        if col in train_df.columns:
            projections = projections.merge(
                train_df[['PlayerID', col]].drop_duplicates(), 
                on='PlayerID', how='left'
            )
    
    # =======================================================
    # ENHANCEMENT 1: Incorporate Starter Status from Lineups
    # =======================================================
    if 'df_lineups' in globals() and not df_lineups.empty:
        print("📋 Incorporating starter status from lineups...")
        
        # Create starter mapping
        starter_players = []
        for _, row in df_lineups.iterrows():
            for i in range(1, 6):
                starter_col = f'starter_{i}'
                if starter_col in row and pd.notna(row[starter_col]):
                    starter_players.append(row[starter_col].strip().lower())
        
        starter_players = set(starter_players)
        projections['Is_Starter'] = projections['Player'].str.lower().isin(starter_players).astype(int)
        
        # Boost projections for starters
        projections['Starter_Boost'] = projections['Is_Starter'] * projections['Base_Projection'] * 0.15
    else:
        projections['Is_Starter'] = 0
        projections['Starter_Boost'] = 0
    
    # =======================================================
    # ENHANCEMENT 2: Incorporate Injury Status from MNP
    # =======================================================
    if 'mnp_df' in globals() and not mnp_df.empty:
        print("🏥 Incorporating injury status from MNP data...")
        injured_players = set(mnp_df['player'].str.lower().dropna())
        projections['Is_Injured'] = projections['Player'].str.lower().isin(injured_players).astype(int)
        
        # Penalize injured players
        projections['Injury_Penalty'] = projections['Is_Injured'] * projections['Base_Projection'] * 0.5
    else:
        projections['Is_Injured'] = 0
        projections['Injury_Penalty'] = 0
    
    # =======================================================
    # ENHANCEMENT 3: Recent Performance Momentum
    # =======================================================
    if 'FPT_roll_mean' in projections.columns:
        # Players trending up get boost
        projections['Momentum_Ratio'] = projections['FPT_roll_mean'] / projections['Base_Projection'].replace(0, 1)
        projections['Momentum_Boost'] = np.where(
            projections['Momentum_Ratio'] > 1.1,
            projections['Base_Projection'] * 0.1,
            np.where(projections['Momentum_Ratio'] < 0.9, 
                    projections['Base_Projection'] * -0.1, 0)
        )
    else:
        projections['Momentum_Boost'] = 0
    
    # =======================================================
    # ENHANCEMENT 4: Usage Rate Integration
    # =======================================================
    if 'Usage_Raw' in projections.columns:
        # Normalize usage and apply boost
        usage_mean = projections['Usage_Raw'].mean()
        projections['Usage_Boost'] = (
            (projections['Usage_Raw'] - usage_mean) / usage_mean * projections['Base_Projection'] * 0.2
        )
    else:
        projections['Usage_Boost'] = 0
    
    # =======================================================
    # ENHANCEMENT 5: Defense vs Position (DvP) Adjustment
    # =======================================================
    if 'Allow_FPT' in projections.columns:
        # Teams that allow more FPT get boost
        dvp_mean = projections['Allow_FPT'].mean()
        projections['DvP_Boost'] = (
            (projections['Allow_FPT'] - dvp_mean) / dvp_mean * projections['Base_Projection'] * 0.15
        )
    else:
        projections['DvP_Boost'] = 0
    
    # =======================================================
    # FINAL ENHANCED PROJECTION
    # =======================================================
    projections['Enhanced_Projection'] = (
        projections['Base_Projection'] + 
        projections['Starter_Boost'] +
        projections['Momentum_Boost'] + 
        projections['Usage_Boost'] +
        projections['DvP_Boost'] -
        projections['Injury_Penalty']
    )
    
    # Ensure projections are reasonable
    projections['Enhanced_Projection'] = projections['Enhanced_Projection'].clip(lower=0)
    
    return projections

# Create enhanced projections
enhanced_projections = create_enhanced_projections()

print(f"✅ Created enhanced projections for {len(enhanced_projections)} players")
print("\n📊 Projection Enhancements Summary:")
enhancement_cols = ['Starter_Boost', 'Momentum_Boost', 'Usage_Boost', 'DvP_Boost', 'Injury_Penalty']
for col in enhancement_cols:
    if col in enhanced_projections.columns:
        avg_effect = enhanced_projections[col].mean()
        print(f"   {col}: {avg_effect:+.2f}")

# Show top players with enhanced projections
top_enhanced = enhanced_projections.nlargest(10, 'Enhanced_Projection')[
    ['Player', 'Pos', 'Team', 'CR', 'Base_Projection', 'Enhanced_Projection', 'Is_Starter', 'Is_Injured']
]
print("\n🏆 TOP 10 ENHANCED PROJECTIONS:")
print(top_enhanced.to_string(index=False))

# =======================================================
# STEP 2: Smart Player Pool Filtering
# =======================================================

def create_optimized_player_pool(projections_df):
    """Create optimized player pool using multiple strategies"""
    
    # Strategy 1: Value-based filtering (Projection per Credit)
    projections_df['Value_Ratio'] = projections_df['Enhanced_Projection'] / projections_df['CR']
    
    # Strategy 2: Remove severely injured players
    filtered = projections_df[
        (projections_df['Is_Injured'] == 0) &  # Remove injured players
        (projections_df['CR'] <= 18) &         # Reasonable cost limit
        (projections_df['Enhanced_Projection'] > 5)  # Minimum projection
    ].copy()
    
    # Strategy 3: Position-specific value thresholds
    pos_value_thresholds = {}
    for pos in ['G', 'F', 'C']:
        pos_players = filtered[filtered['Pos'] == pos]
        if len(pos_players) > 0:
            threshold = pos_players['Value_Ratio'].quantile(0.3)  # Keep top 70% by value
            pos_value_thresholds[pos] = threshold
    
    # Apply position-specific filtering
    keep_mask = pd.Series(False, index=filtered.index)
    for pos, threshold in pos_value_thresholds.items():
        pos_mask = (filtered['Pos'] == pos) & (filtered['Value_Ratio'] >= threshold)
        keep_mask = keep_mask | pos_mask
    
    filtered = filtered[keep_mask].copy()
    
    return filtered

optimized_pool = create_optimized_player_pool(enhanced_projections)

print(f"\n🎯 OPTIMIZED PLAYER POOL:")
print(f"   Total players: {len(optimized_pool)}")
print(f"   Guards: {len(optimized_pool[optimized_pool['Pos'] == 'G'])}")
print(f"   Forwards: {len(optimized_pool[optimized_pool['Pos'] == 'F'])}")
print(f"   Centers: {len(optimized_pool[optimized_pool['Pos'] == 'C'])}")

# =======================================================
# STEP 3: Improved Lineup Optimization
# =======================================================

# Euroleague Fantasy constraints
MAX_BUDGET = 100.0
TOTAL_PLAYERS = 10
POSITION_RULES = {
    'G': {'min': 3, 'max': 5, 'ideal': 4},
    'F': {'min': 3, 'max': 5, 'ideal': 4}, 
    'C': {'min': 2, 'max': 3, 'ideal': 2}
}

print(f"\n{'='*60}")
print("OPTIMIZED INTEGER LINEAR PROGRAMMING SOLUTION")
print(f"{'='*60}")

try:
    # Create the optimization problem
    prob = LpProblem("Euroleague_Fantasy_Optimization", LpMaximize)
    
    # Decision variables
    player_vars = LpVariable.dicts("Player", optimized_pool.index, cat=LpBinary)
    
    # Objective: Maximize enhanced projections
    prob += lpSum(
        optimized_pool.loc[i, "Enhanced_Projection"] * player_vars[i] 
        for i in optimized_pool.index
    )
    
    # Total players constraint
    prob += lpSum(player_vars[i] for i in optimized_pool.index) == TOTAL_PLAYERS
    
    # Budget constraint
    prob += lpSum(
        optimized_pool.loc[i, "CR"] * player_vars[i] 
        for i in optimized_pool.index
    ) <= MAX_BUDGET
    
    # Position constraints
    for pos, rules in POSITION_RULES.items():
        pos_players = optimized_pool[optimized_pool["Pos"] == pos].index
        prob += lpSum(player_vars[i] for i in pos_players) >= rules['min']
        prob += lpSum(player_vars[i] for i in pos_players) <= rules['max']
    
    # Solve
    prob.solve()
    
    if LpStatus[prob.status] == "Optimal":
        # Get selected players
        selected_indices = [i for i in optimized_pool.index if player_vars[i].value() == 1]
        optimal_team = optimized_pool.loc[selected_indices].copy()
        
        total_score = optimal_team["Enhanced_Projection"].sum()
        total_budget = optimal_team["CR"].sum()
        pos_counts = optimal_team["Pos"].value_counts().to_dict()
        
        print("🎉 OPTIMAL LINEUP FOUND!")
        print(f"   Enhanced Score: {total_score:.2f}")
        print(f"   Budget Used: {total_budget:.1f}/{MAX_BUDGET}")
        print(f"   Position Distribution: {pos_counts}")
        
        # Display optimal team
        optimal_team_sorted = optimal_team.sort_values(["Pos", "Enhanced_Projection"], ascending=[True, False])
        display_cols = ["Player", "Pos", "Team", "CR", "Base_Projection", "Enhanced_Projection", "Is_Starter"]
        display_cols = [col for col in display_cols if col in optimal_team_sorted.columns]
        
        print("\n🏀 OPTIMAL LINEUP:")
        display(optimal_team_sorted[display_cols])
        
        # Value analysis
        print(f"\n💰 VALUE ANALYSIS:")
        print(f"   Average Value Ratio: {optimal_team['Value_Ratio'].mean():.3f}")
        print(f"   Total Base Projection: {optimal_team['Base_Projection'].sum():.2f}")
        print(f"   Total Enhancement: {total_score - optimal_team['Base_Projection'].sum():.2f}")
        
    else:
        print(f"❌ No optimal solution found. Status: {LpStatus[prob.status]}")
        
except Exception as e:
    print(f"❌ ILP solver failed: {e}")

# =======================================================
# STEP 4: Alternative High-Risk/High-Reward Lineups
# =======================================================

print(f"\n{'='*60}")
print("ALTERNATIVE STRATEGY LINEUPS")
print(f"{'='*60}")

def generate_alternative_lineups(strategy, player_pool, num_lineups=3):
    """Generate alternative lineups based on different strategies"""
    
    lineups = []
    
    if strategy == "value":
        # Focus on best value players
        sorted_pool = player_pool.sort_values("Value_Ratio", ascending=False)
    elif strategy == "stars_and_scrubs":
        # Mix of expensive stars and cheap value
        expensive = player_pool[player_pool["CR"] >= 14].sort_values("Enhanced_Projection", ascending=False)
        cheap = player_pool[player_pool["CR"] <= 9].sort_values("Value_Ratio", ascending=False)
        sorted_pool = pd.concat([expensive.head(10), cheap.head(20)]).sample(frac=1)
    elif strategy == "safety":
        # Focus on starters and consistent performers
        if 'Is_Starter' in player_pool.columns:
            sorted_pool = player_pool[player_pool['Is_Starter'] == 1].sort_values("Enhanced_Projection", ascending=False)
        else:
            sorted_pool = player_pool.sort_values("FPT_roll_mean", ascending=False)
    else:
        sorted_pool = player_pool.sort_values("Enhanced_Projection", ascending=False)
    
    for attempt in range(1000):
        if len(lineups) >= num_lineups:
            break
            
        team = []
        budget = 0
        pos_counts = {"G": 0, "F": 0, "C": 0}
        
        for _, player in sorted_pool.iterrows():
            pos = player["Pos"]
            cost = player["CR"]
            
            if (cost + budget <= MAX_BUDGET and 
                pos_counts[pos] < POSITION_RULES[pos]['max'] and
                len(team) < TOTAL_PLAYERS):
                
                team.append(player)
                budget += cost
                pos_counts[pos] += 1
                
                if len(team) == TOTAL_PLAYERS:
                    # Check minimum position requirements
                    if all(pos_counts[p] >= POSITION_RULES[p]['min'] for p in POSITION_RULES):
                        team_df = pd.DataFrame(team)
                        score = team_df["Enhanced_Projection"].sum()
                        lineups.append((score, budget, pos_counts.copy(), team_df))
                    break
    
    return lineups

# Generate alternative lineups
strategies = {
    "💰 Value Focus": "value",
    "⭐ Stars & Scrubs": "stars_and_scrubs", 
    "🛡️ Safe Plays": "safety"
}

for strategy_name, strategy_type in strategies.items():
    print(f"\n{strategy_name}:")
    alt_lineups = generate_alternative_lineups(strategy_type, optimized_pool, 1)
    
    if alt_lineups:
        score, budget, pos_counts, team_df = alt_lineups[0]
        team_df = team_df.sort_values(["Pos", "Enhanced_Projection"], ascending=[True, False])
        
        print(f"   Score: {score:.2f} | Budget: {budget:.1f} | Positions: {pos_counts}")
        display_cols = ["Player", "Pos", "Team", "CR", "Enhanced_Projection"]
        display_cols = [col for col in display_cols if col in team_df.columns]
        display(team_df[display_cols].head(8))
    else:
        print("   No valid lineup found")

# =======================================================
# STEP 5: Export Final Recommendations
# =======================================================

print(f"\n{'='*60}")
print("FINAL RECOMMENDATIONS")
print(f"{'='*60}")

# Top value picks by position
print("\n💎 TOP VALUE PICKS BY POSITION:")
for pos in ['G', 'F', 'C']:
    pos_players = optimized_pool[optimized_pool['Pos'] == pos]
    if len(pos_players) > 0:
        top_value = pos_players.nlargest(3, 'Value_Ratio')[
            ['Player', 'Team', 'CR', 'Enhanced_Projection', 'Value_Ratio']
        ]
        print(f"\n{pos} - Best Value:")
        for _, player in top_value.iterrows():
            print(f"   {player['Player']} | {player['Team']} | CR: {player['CR']} | "
                  f"Proj: {player['Enhanced_Projection']:.1f} | Value: {player['Value_Ratio']:.3f}")

# Players to avoid
if 'Is_Injured' in optimized_pool.columns:
    injured_to_avoid = enhanced_projections[
        (enhanced_projections['Is_Injured'] == 1) & 
        (enhanced_projections['Base_Projection'] > 15)  # Only mention significant players
    ]
    if len(injured_to_avoid) > 0:
        print(f"\n🚫 INJURED PLAYERS TO AVOID:")
        for _, player in injured_to_avoid.head(5).iterrows():
            print(f"   {player['Player']} | {player['Team']} | Projection: {player['Enhanced_Projection']:.1f}")

print(f"\n✅ OPTIMIZED LINEUP GENERATION COMPLETE!")
print("   Your lineup now incorporates:")
print("   • Model predictions with feature importance")
print("   • Starter status from actual lineups") 
print("   • Injury status from MNP data")
print("   • Recent performance momentum")
print("   • Usage rate and DvP adjustments")
print("   • Multiple optimization strategies")

🚀 OPTIMIZED Lineup Generation with Model Integration
📋 Incorporating starter status from lineups...
🏥 Incorporating injury status from MNP data...
✅ Created enhanced projections for 280 players

📊 Projection Enhancements Summary:
   Starter_Boost: +1.96
   Momentum_Boost: -0.00
   Usage_Boost: +6.34
   DvP_Boost: -0.05
   Injury_Penalty: +1.11

🏆 TOP 10 ENHANCED PROJECTIONS:
            Player Pos  Team   CR  Base_Projection  Enhanced_Projection  Is_Starter  Is_Injured
       S. Vezenkov   F   NaN 17.0       181.432471           266.939034           1           0
           N. Hifi   G   NaN 13.9       173.953271           265.926092           0           0
       F. Petrusev   F   NaN 15.4       180.043941           253.480208           1           0
      S. Francisco   G   NaN 16.6       162.749542           243.973042           1           0
      N. Milutinov   C   NaN 16.0       181.052550           232.346920           1           0
         C. Moneke   F   NaN 16.1       179.71

,Player,Pos,Team,CR,Base_Projection,Enhanced_Projection,Is_Starter
222,M. Ndiaye,C,NaN,7.5,87.246952,91.041027,0
228,L. Birutis,C,NaN,6.2,66.970279,67.169656,0
156,F. Petrusev,F,NaN,15.4,180.043941,253.480208,1
165,T. Luwawu-cabarrot,F,NaN,12.0,153.930785,207.995540,0
142,I. Mike,F,NaN,9.0,122.725605,145.727864,0
60,N. Hifi,G,NaN,13.9,173.953271,265.926092,0
0,T. Dorsey,G,NaN,11.5,132.329402,178.521697,0
69,O. Moore,G,NaN,11.2,132.149108,168.037379,0
30,A. Obst,G,NaN,8.2,121.416689,147.333291,0
59,S. Herrera,G,NaN,5.1,69.877093,88.258862,1



💰 VALUE ANALYSIS:
   Average Value Ratio: 15.789
   Total Base Projection: 1240.64
   Total Enhancement: 372.85

ALTERNATIVE STRATEGY LINEUPS

💰 Value Focus:
   No valid lineup found

⭐ Stars & Scrubs:
   Score: 1089.53 | Budget: 89.0 | Positions: {'G': 3, 'F': 4, 'C': 3}


,Player,Pos,Team,CR,Enhanced_Projection
222,M. Ndiaye,C,NaN,7.5,91.041027
226,D. Motiejunas,C,NaN,7.5,87.257162
228,L. Birutis,C,NaN,6.2,67.169656
125,C. Moneke,F,NaN,16.1,225.106230
183,N. Reuvers,F,NaN,7.7,89.534923
132,S. Niang,F,NaN,8.8,88.355072
172,G. Ricci,F,NaN,5.3,53.023992
4,M. James,G,NaN,16.5,214.925287



🛡️ Safe Plays:
   No valid lineup found

FINAL RECOMMENDATIONS

💎 TOP VALUE PICKS BY POSITION:

G - Best Value:
   N. Hifi | nan | CR: 13.9 | Proj: 265.9 | Value: 19.131
   A. Obst | nan | CR: 8.2 | Proj: 147.3 | Value: 17.967
   S. Herrera | nan | CR: 5.1 | Proj: 88.3 | Value: 17.306

F - Best Value:
   T. Luwawu-cabarrot | nan | CR: 12.0 | Proj: 208.0 | Value: 17.333
   F. Petrusev | nan | CR: 15.4 | Proj: 253.5 | Value: 16.460
   I. Mike | nan | CR: 9.0 | Proj: 145.7 | Value: 16.192

C - Best Value:
   N. Milutinov | nan | CR: 16.0 | Proj: 232.3 | Value: 14.522
   M. Kabengele | nan | CR: 13.8 | Proj: 199.3 | Value: 14.440
   M. Wright | nan | CR: 12.5 | Proj: 175.6 | Value: 14.048

🚫 INJURED PLAYERS TO AVOID:
   S. Miljenovic | nan | Projection: 5.5
   X. Rathan-mayes | nan | Projection: 29.7
   S. Wilbekin | nan | Projection: 18.4
   S. Mckissic | nan | Projection: 6.0
   N. Williams-goss | nan | Projection: 36.9

✅ OPTIMIZED LINEUP GENERATION COMPLETE!
   Your lineup now incorpo

In [13]:
# =======================================================
# Cell 10 - Euroleague Fantasy Rules Implementation (Fixed Budget)
# =======================================================
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus
import os
from datetime import datetime

print("🎯 EUROLEAGUE FANTASY RULES IMPLEMENTATION")
print("===========================================")

# Euroleague Fantasy Specific Rules
MAX_BUDGET = 101.0
TOTAL_PLAYERS = 10
CAPTAIN_MULTIPLIER = 2.0

print(f"📋 FANTASY RULES:")
print(f"   • Total Budget: {MAX_BUDGET} credits")
print(f"   • Total Players: {TOTAL_PLAYERS}")
print(f"   • 6 Active Players (5 starters + 6th player)")
print(f"   • 1 Captain (×{CAPTAIN_MULTIPLIER} points)")
print(f"   • 4 Substitutes")
print(f"   • ALL 10 PLAYERS MUST FIT IN {MAX_BUDGET} CREDITS")

# =======================================================
# STEP 1: Load the enhanced projections
# =======================================================

# Use the enhanced projections from Cell 9
if 'enhanced_projections' not in globals():
    print("❌ Enhanced projections not found. Please run Cell 9 first.")
else:
    print(f"✅ Loaded {len(enhanced_projections)} player projections")
    
    # Filter valid players for optimization
    valid_players = enhanced_projections[
        (enhanced_projections['Is_Injured'] == 0) &
        (enhanced_projections['CR'] <= 16) &
        (enhanced_projections['Enhanced_Projection'] > 5)
    ].copy()
    
    print(f"✅ {len(valid_players)} valid players for optimization")

# =======================================================
# STEP 2: Optimize ALL 10 Players Together with Captain
# =======================================================

def optimize_complete_team_with_captain(players_df, budget=100):
    """Optimize ALL 10 players (6 active + 4 substitutes) with captain within budget"""
    
    print(f"\n🔄 Optimizing Complete Team of 10 Players...")
    print(f"   Total Budget: {budget} credits")
    
    # Create the optimization problem
    prob = LpProblem("Euroleague_Complete_Team", LpMaximize)
    
    # Decision variables
    player_vars = LpVariable.dicts("Player", players_df.index, cat=LpBinary)
    captain_vars = LpVariable.dicts("Captain", players_df.index, cat=LpBinary)
    active_vars = LpVariable.dicts("Active", players_df.index, cat=LpBinary)  # 6 active players
    
    # Objective: Maximize total points (captain gets double, all players count)
    prob += lpSum(
        players_df.loc[i, "Enhanced_Projection"] * player_vars[i] +  # All players
        players_df.loc[i, "Enhanced_Projection"] * captain_vars[i]   # Captain bonus
        for i in players_df.index
    )
    
    # Constraints
    # Total players: 10
    prob += lpSum(player_vars[i] for i in players_df.index) == 10
    
    # Exactly 6 active players (starters + 6th)
    prob += lpSum(active_vars[i] for i in players_df.index) == 6
    
    # Active players must be selected players
    for i in players_df.index:
        prob += active_vars[i] <= player_vars[i]
    
    # Only one captain
    prob += lpSum(captain_vars[i] for i in players_df.index) == 1
    
    # Captain must be one of the active players
    for i in players_df.index:
        prob += captain_vars[i] <= active_vars[i]
    
    # Budget constraint - ALL 10 PLAYERS
    prob += lpSum(
        players_df.loc[i, "CR"] * player_vars[i] 
        for i in players_df.index
    ) <= budget
    
    # Position constraints for active players (6 players)
    guards = players_df[players_df["Pos"] == "G"].index
    forwards = players_df[players_df["Pos"] == "F"].index
    centers = players_df[players_df["Pos"] == "C"].index
    
    # At least 2G, 2F, 1C in the 6 active players
    prob += lpSum(active_vars[i] for i in guards) >= 2
    prob += lpSum(active_vars[i] for i in forwards) >= 2
    prob += lpSum(active_vars[i] for i in centers) >= 1
    
    # Position constraints for substitutes (4 players)
    # No specific position requirements for substitutes
    
    # Solve
    prob.solve()
    
    if LpStatus[prob.status] == "Optimal":
        # Get all selected players
        selected_indices = [i for i in players_df.index if player_vars[i].value() == 1]
        all_players = players_df.loc[selected_indices].copy()
        
        # Identify active players
        active_indices = [i for i in players_df.index if active_vars[i].value() == 1]
        active_players = players_df.loc[active_indices].copy()
        
        # Identify substitutes
        substitute_indices = [i for i in selected_indices if i not in active_indices]
        substitutes = players_df.loc[substitute_indices].copy()
        
        # Identify captain
        captain_index = [i for i in players_df.index if captain_vars[i].value() == 1][0]
        captain = players_df.loc[captain_index]
        
        # Calculate total score
        base_score = all_players["Enhanced_Projection"].sum()
        captain_bonus = captain["Enhanced_Projection"]  # Already counted once, so add once more
        total_score = base_score + captain_bonus
        
        total_budget = all_players["CR"].sum()
        
        print("✅ COMPLETE TEAM OPTIMIZED!")
        print(f"   Total Score: {total_score:.2f} (Base: {base_score:.2f} + Captain: {captain_bonus:.2f})")
        print(f"   Budget Used: {total_budget:.1f}/{budget}")
        print(f"   Budget Remaining: {budget - total_budget:.1f}")
        print(f"   Active Players: {len(active_players)}")
        print(f"   Substitutes: {len(substitutes)}")
        print(f"   Captain: {captain['Player']} ({captain['Pos']}) - {captain['Enhanced_Projection']:.2f} × {CAPTAIN_MULTIPLIER}")
        
        return all_players, active_players, substitutes, captain, total_score, total_budget
    else:
        print(f"❌ No optimal solution found. Status: {LpStatus[prob.status]}")
        return None, None, None, None, 0, 0

# Optimize complete team
all_players, active_players, substitutes, captain, total_score, total_budget = optimize_complete_team_with_captain(valid_players, MAX_BUDGET)

# =======================================================
# STEP 3: Display and Export Results
# =======================================================

print(f"\n{'='*60}")
print("FINAL EUROLEAGUE FANTASY TEAM")
print(f"{'='*60}")

if all_players is not None:
    # Create final team DataFrame with roles
    active_display = active_players[['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
    active_display['Role'] = 'Active'
    active_display.loc[active_display['Player'] == captain['Player'], 'Role'] = 'CAPTAIN'
    
    sub_display = substitutes[['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
    sub_display['Role'] = 'Substitute'
    
    final_team = pd.concat([active_display, sub_display])
    
    # Calculate detailed scores
    active_base_score = active_players["Enhanced_Projection"].sum()
    sub_score = substitutes["Enhanced_Projection"].sum()
    captain_bonus = captain["Enhanced_Projection"]
    
    print(f"📊 TEAM BREAKDOWN:")
    print(f"   Active Players (6): {active_base_score:.2f} points")
    print(f"   Captain Bonus: +{captain_bonus:.2f} points")
    print(f"   Substitutes (4): {sub_score:.2f} points")
    print(f"   TOTAL SCORE: {total_score:.2f} points")
    print(f"   TOTAL BUDGET: {total_budget:.1f}/{MAX_BUDGET} credits")
    print(f"   REMAINING: {MAX_BUDGET - total_budget:.1f} credits (for coach)")
    
    print(f"\n🏀 ACTIVE PLAYERS (Starters + 6th):")
    active_sorted = active_display.sort_values(['Role', 'Enhanced_Projection'], ascending=[False, False])
    display(active_sorted)
    
    print(f"\n🔄 SUBSTITUTES:")
    sub_sorted = sub_display.sort_values('Enhanced_Projection', ascending=False)
    display(sub_sorted)
    
    print(f"\n💰 BUDGET DISTRIBUTION:")
    budget_summary = final_team.groupby('Role').agg({
        'CR': 'sum',
        'Enhanced_Projection': 'sum',
        'Player': 'count'
    }).rename(columns={'Player': 'Count', 'Enhanced_Projection': 'Total_Projection'})
    print(budget_summary)
    
    # Export final team
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"euroleague_complete_team_{timestamp}.csv"
    filepath = os.path.join("Euroleague", filename)
    final_team.to_csv(filepath, index=False)
    print(f"\n💾 Complete team saved to: {filepath}")
    
    print(f"\n🎯 STRATEGY SUMMARY:")
    print(f"   • All 10 players fit within {MAX_BUDGET} credit budget")
    print(f"   • Captain gets {CAPTAIN_MULTIPLIER}× points")
    print(f"   • 6 active players optimized for maximum points")
    print(f"   • 4 substitutes provide bench coverage")
    print(f"   • {MAX_BUDGET - total_budget:.1f} credits remaining for coach")

else:
    print("❌ Team optimization failed. Please check the constraints.")

print(f"\n✨ EUROLEAGUE FANTASY TEAM BUILDING COMPLETE!")

🎯 EUROLEAGUE FANTASY RULES IMPLEMENTATION
📋 FANTASY RULES:
   • Total Budget: 101.0 credits
   • Total Players: 10
   • 6 Active Players (5 starters + 6th player)
   • 1 Captain (×2.0 points)
   • 4 Substitutes
   • ALL 10 PLAYERS MUST FIT IN 101.0 CREDITS
✅ Loaded 280 player projections
✅ 216 valid players for optimization

🔄 Optimizing Complete Team of 10 Players...
   Total Budget: 101.0 credits
✅ COMPLETE TEAM OPTIMIZED!
   Total Score: 1906.13 (Base: 1640.21 + Captain: 265.93)
   Budget Used: 101.0/101.0
   Budget Remaining: 0.0
   Active Players: 6
   Substitutes: 4
   Captain: N. Hifi (G) - 265.93 × 2.0

FINAL EUROLEAGUE FANTASY TEAM
📊 TEAM BREAKDOWN:
   Active Players (6): 1102.48 points
   Captain Bonus: +265.93 points
   Substitutes (4): 537.73 points
   TOTAL SCORE: 1906.13 points
   TOTAL BUDGET: 101.0/101.0 credits
   REMAINING: 0.0 credits (for coach)

🏀 ACTIVE PLAYERS (Starters + 6th):


,Player,Pos,Team,CR,Enhanced_Projection,Role
60,N. Hifi,G,NaN,13.9,265.926092,CAPTAIN
156,F. Petrusev,F,NaN,15.4,253.480208,Active
0,T. Dorsey,G,NaN,11.5,178.521697,Active
162,H. Diallo,F,NaN,11.0,167.778490,Active
142,I. Mike,F,NaN,9.0,145.727864,Active
222,M. Ndiaye,C,NaN,7.5,91.041027,Active



🔄 SUBSTITUTES:


,Player,Pos,Team,CR,Enhanced_Projection,Role
165,T. Luwawu-cabarrot,F,NaN,12.0,207.995540,Substitute
30,A. Obst,G,NaN,8.2,147.333291,Substitute
38,T. Blatt,G,NaN,7.4,94.142451,Substitute
59,S. Herrera,G,NaN,5.1,88.258862,Substitute



💰 BUDGET DISTRIBUTION:
              CR  Total_Projection  Count
Role                                     
Active      54.4        836.549287      5
CAPTAIN     13.9        265.926092      1
Substitute  32.7        537.730144      4

💾 Complete team saved to: Euroleague\euroleague_complete_team_20251126_170130.csv

🎯 STRATEGY SUMMARY:
   • All 10 players fit within 101.0 credit budget
   • Captain gets 2.0× points
   • 6 active players optimized for maximum points
   • 4 substitutes provide bench coverage
   • 0.0 credits remaining for coach

✨ EUROLEAGUE FANTASY TEAM BUILDING COMPLETE!


In [14]:
# =======================================================
# Cell 10 - Euroleague Fantasy Rules with Updated Position Constraints
# =======================================================
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus
import os
from datetime import datetime

print("🎯 EUROLEAGUE FANTASY RULES IMPLEMENTATION")
print("===========================================")

# Euroleague Fantasy Specific Rules
MAX_BUDGET = 100.0
TOTAL_PLAYERS = 10
CAPTAIN_MULTIPLIER = 2.0
NUM_LINEUPS = 5  # Number of top lineups to generate

# UPDATED POSITION CONSTRAINTS
MIN_GUARDS = 3
MIN_FORWARDS = 3
MIN_CENTERS = 2

print(f"📋 FANTASY RULES:")
print(f"   • Total Budget: {MAX_BUDGET} credits")
print(f"   • Total Players: {TOTAL_PLAYERS}")
print(f"   • 6 Active Players (5 starters + 6th player)")
print(f"   • 1 Captain (×{CAPTAIN_MULTIPLIER} points)")
print(f"   • 4 Substitutes")
print(f"   • Position Requirements: At least {MIN_GUARDS}G, {MIN_FORWARDS}F, {MIN_CENTERS}C")
print(f"   • Generating Top {NUM_LINEUPS} Lineups")

# =======================================================
# STEP 1: Load the enhanced projections
# =======================================================

# Use the enhanced projections from Cell 9
if 'enhanced_projections' not in globals():
    print("❌ Enhanced projections not found. Please run Cell 9 first.")
else:
    print(f"✅ Loaded {len(enhanced_projections)} player projections")
    
    # Filter valid players for optimization
    valid_players = enhanced_projections[
        (enhanced_projections['Is_Injured'] == 0) &
        (enhanced_projections['CR'] <= 16) &
        (enhanced_projections['Enhanced_Projection'] > 5)
    ].copy()
    
    print(f"✅ {len(valid_players)} valid players for optimization")

# =======================================================
# STEP 2: Generate Multiple Optimal Lineups with Updated Position Constraints
# =======================================================

def generate_top_lineups(players_df, budget=100, num_lineups=5):
    """Generate multiple optimal lineups with updated position constraints"""
    
    print(f"\n🔄 Generating Top {num_lineups} Lineups...")
    print(f"   Position Requirements: {MIN_GUARDS}G, {MIN_FORWARDS}F, {MIN_CENTERS}C")
    
    lineups = []
    seen_teams = set()
    
    for lineup_num in range(num_lineups):
        print(f"   Generating lineup {lineup_num + 1}/{num_lineups}...")
        
        # Create the optimization problem
        prob = LpProblem(f"Euroleague_Lineup_{lineup_num}", LpMaximize)
        
        # Decision variables
        player_vars = LpVariable.dicts("Player", players_df.index, cat=LpBinary)
        captain_vars = LpVariable.dicts("Captain", players_df.index, cat=LpBinary)
        active_vars = LpVariable.dicts("Active", players_df.index, cat=LpBinary)
        
        # Objective: Maximize total points
        prob += lpSum(
            players_df.loc[i, "Enhanced_Projection"] * player_vars[i] +
            players_df.loc[i, "Enhanced_Projection"] * captain_vars[i]
            for i in players_df.index
        )
        
        # Basic constraints
        prob += lpSum(player_vars[i] for i in players_df.index) == 10
        prob += lpSum(active_vars[i] for i in players_df.index) == 6
        prob += lpSum(captain_vars[i] for i in players_df.index) == 1
        
        for i in players_df.index:
            prob += active_vars[i] <= player_vars[i]
            prob += captain_vars[i] <= active_vars[i]
        
        # Budget constraint
        prob += lpSum(players_df.loc[i, "CR"] * player_vars[i] for i in players_df.index) <= budget
        
        # UPDATED POSITION CONSTRAINTS FOR ALL 10 PLAYERS
        guards = players_df[players_df["Pos"] == "G"].index
        forwards = players_df[players_df["Pos"] == "F"].index
        centers = players_df[players_df["Pos"] == "C"].index
        
        # Minimum requirements for entire team (10 players)
        prob += lpSum(player_vars[i] for i in guards) >= MIN_GUARDS
        prob += lpSum(player_vars[i] for i in forwards) >= MIN_FORWARDS
        prob += lpSum(player_vars[i] for i in centers) >= MIN_CENTERS
        
        # Position constraints for active players (6 players)
        # At least 2G, 2F, 1C in the 6 active players
        prob += lpSum(active_vars[i] for i in guards) >= 2
        prob += lpSum(active_vars[i] for i in forwards) >= 2
        prob += lpSum(active_vars[i] for i in centers) >= 1
        
        # Avoid duplicate lineups (for lineups after the first)
        if lineup_num > 0:
            # Add constraint to avoid previous lineups
            for prev_lineup in lineups:
                prev_players = set(prev_lineup['all_players'].index)
                prob += lpSum(player_vars[i] for i in prev_players) <= 9  # Force at least 1 different player
        
        # Solve
        prob.solve()
        
        if LpStatus[prob.status] == "Optimal":
            # Get all selected players
            selected_indices = [i for i in players_df.index if player_vars[i].value() == 1]
            all_players = players_df.loc[selected_indices].copy()
            
            # Identify active players
            active_indices = [i for i in players_df.index if active_vars[i].value() == 1]
            active_players = players_df.loc[active_indices].copy()
            
            # Identify substitutes
            substitute_indices = [i for i in selected_indices if i not in active_indices]
            substitutes = players_df.loc[substitute_indices].copy()
            
            # Identify captain
            captain_index = [i for i in players_df.index if captain_vars[i].value() == 1][0]
            captain = players_df.loc[captain_index]
            
            # Calculate scores
            base_score = all_players["Enhanced_Projection"].sum()
            captain_bonus = captain["Enhanced_Projection"]
            total_score = base_score + captain_bonus
            total_budget = all_players["CR"].sum()
            
            # Create team signature to avoid duplicates
            team_signature = frozenset(all_players['Player'].values)
            
            if team_signature not in seen_teams:
                seen_teams.add(team_signature)
                
                lineup_data = {
                    'lineup_num': lineup_num + 1,
                    'all_players': all_players,
                    'active_players': active_players,
                    'substitutes': substitutes,
                    'captain': captain,
                    'total_score': total_score,
                    'total_budget': total_budget,
                    'base_score': base_score,
                    'captain_bonus': captain_bonus
                }
                lineups.append(lineup_data)
                
                print(f"      ✅ Lineup {lineup_num + 1}: {total_score:.2f} points, {total_budget:.1f} credits")
                
                # Show position distribution
                pos_counts = all_players['Pos'].value_counts()
                print(f"         Positions: G:{pos_counts.get('G', 0)} F:{pos_counts.get('F', 0)} C:{pos_counts.get('C', 0)}")
            else:
                print(f"      ⚠️  Duplicate lineup found, skipping...")
        else:
            print(f"      ❌ No solution found for lineup {lineup_num + 1}")
            break
    
    return lineups

# Generate multiple lineups
top_lineups = generate_top_lineups(valid_players, MAX_BUDGET, NUM_LINEUPS)

# =======================================================
# STEP 3: Display All Top Lineups
# =======================================================

print(f"\n{'='*60}")
print(f"TOP {len(top_lineups)} EUROLEAGUE FANTASY LINEUPS")
print(f"POSITION REQUIREMENTS: {MIN_GUARDS}G, {MIN_FORWARDS}F, {MIN_CENTERS}C")
print(f"{'='*60}")

for lineup_data in top_lineups:
    lineup_num = lineup_data['lineup_num']
    all_players = lineup_data['all_players']
    active_players = lineup_data['active_players']
    substitutes = lineup_data['substitutes']
    captain = lineup_data['captain']
    total_score = lineup_data['total_score']
    total_budget = lineup_data['total_budget']
    
    print(f"\n🏆 LINEUP #{lineup_num}")
    print(f"   📊 Score: {total_score:.2f} | Budget: {total_budget:.1f}/{MAX_BUDGET} | Remaining: {MAX_BUDGET - total_budget:.1f}")
    print(f"   ⭐ Captain: {captain['Player']} ({captain['Pos']}) - {captain['Enhanced_Projection']:.2f} × {CAPTAIN_MULTIPLIER}")
    
    # Create display DataFrames
    active_display = active_players[['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
    active_display['Role'] = 'Active'
    active_display.loc[active_display['Player'] == captain['Player'], 'Role'] = 'CAPTAIN'
    
    sub_display = substitutes[['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
    sub_display['Role'] = 'Substitute'
    
    print(f"\n   🏀 ACTIVE PLAYERS:")
    active_sorted = active_display.sort_values(['Role', 'Enhanced_Projection'], ascending=[False, False])
    print(active_sorted.to_string(index=False))
    
    print(f"\n   🔄 SUBSTITUTES:")
    sub_sorted = sub_display.sort_values('Enhanced_Projection', ascending=False)
    print(sub_sorted.to_string(index=False))
    
    # Position distribution
    pos_counts = all_players['Pos'].value_counts()
    print(f"\n   📋 POSITION DISTRIBUTION: G:{pos_counts.get('G', 0)} F:{pos_counts.get('F', 0)} C:{pos_counts.get('C', 0)}")
    
    # Verify position constraints are met
    g_count = pos_counts.get('G', 0)
    f_count = pos_counts.get('F', 0)
    c_count = pos_counts.get('C', 0)
    
    constraints_met = (g_count >= MIN_GUARDS and f_count >= MIN_FORWARDS and c_count >= MIN_CENTERS)
    status = "✅" if constraints_met else "❌"
    print(f"   {status} POSITION REQUIREMENTS: {MIN_GUARDS}G/{MIN_FORWARDS}F/{MIN_CENTERS}C - ACTUAL: {g_count}G/{f_count}F/{c_count}C")
    
    print("-" * 50)

# =======================================================
# STEP 4: Compare Top Lineups
# =======================================================

print(f"\n{'='*60}")
print("LINEUP COMPARISON SUMMARY")
print(f"POSITION REQUIREMENTS: {MIN_GUARDS}G, {MIN_FORWARDS}F, {MIN_CENTERS}C")
print(f"{'='*60}")

if top_lineups:
    # Create comparison table
    comparison_data = []
    for lineup in top_lineups:
        lineup_num = lineup['lineup_num']
        captain_name = lineup['captain']['Player']
        captain_pos = lineup['captain']['Pos']
        total_score = lineup['total_score']
        total_budget = lineup['total_budget']
        
        # Count positions
        pos_counts = lineup['all_players']['Pos'].value_counts()
        g_count = pos_counts.get('G', 0)
        f_count = pos_counts.get('F', 0)
        c_count = pos_counts.get('C', 0)
        
        # Get top 3 players by projection
        top_players = lineup['all_players'].nlargest(3, 'Enhanced_Projection')['Player'].tolist()
        top_players_str = ", ".join(top_players[:2])  # Show top 2 for brevity
        
        comparison_data.append({
            'Lineup': lineup_num,
            'Score': f"{total_score:.1f}",
            'Budget': f"{total_budget:.1f}",
            'Remaining': f"{MAX_BUDGET - total_budget:.1f}",
            'Captain': f"{captain_name} ({captain_pos})",
            'G/F/C': f"{g_count}/{f_count}/{c_count}",
            'Top Players': top_players_str
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df['Score_Num'] = comparison_df['Score'].astype(float)
    comparison_df = comparison_df.sort_values('Score_Num', ascending=False)
    comparison_df = comparison_df.drop('Score_Num', axis=1)
    
    print("🏆 TOP LINEUPS RANKED BY SCORE:")
    print(comparison_df.to_string(index=False))
    
    # Export all lineups
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    for lineup in top_lineups:
        lineup_num = lineup['lineup_num']
        
        # Combine active players and substitutes
        active_display = lineup['active_players'][['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
        active_display['Role'] = 'Active'
        active_display.loc[active_display['Player'] == lineup['captain']['Player'], 'Role'] = 'CAPTAIN'
        
        sub_display = lineup['substitutes'][['Player', 'Pos', 'Team', 'CR', 'Enhanced_Projection']].copy()
        sub_display['Role'] = 'Substitute'
        
        final_team = pd.concat([active_display, sub_display])
        
        # Export individual lineup
        filename = f"euroleague_lineup_{MIN_GUARDS}G{MIN_FORWARDS}F{MIN_CENTERS}C_{lineup_num}_{timestamp}.csv"
        filepath = os.path.join("Euroleague", filename)
        final_team.to_csv(filepath, index=False)
    
    print(f"\n💾 All {len(top_lineups)} lineups saved to Euroleague folder")
    
    # Show strategy insights
    print(f"\n🎯 STRATEGY INSIGHTS:")
    print(f"   • Best lineup scores {top_lineups[0]['total_score']:.2f} points")
    print(f"   • All lineups meet position requirements: {MIN_GUARDS}G, {MIN_FORWARDS}F, {MIN_CENTERS}C")
    print(f"   • Budget usage ranges from {min([l['total_budget'] for l in top_lineups]):.1f} to {max([l['total_budget'] for l in top_lineups]):.1f} credits")
    print(f"   • {len(set([l['captain']['Player'] for l in top_lineups]))} different captains across lineups")
    
    # Position distribution analysis
    all_pos_counts = []
    for lineup in top_lineups:
        pos_counts = lineup['all_players']['Pos'].value_counts()
        all_pos_counts.append({
            'G': pos_counts.get('G', 0),
            'F': pos_counts.get('F', 0),
            'C': pos_counts.get('C', 0)
        })
    
    avg_g = np.mean([p['G'] for p in all_pos_counts])
    avg_f = np.mean([p['F'] for p in all_pos_counts])
    avg_c = np.mean([p['C'] for p in all_pos_counts])
    
    print(f"   • Average position distribution: {avg_g:.1f}G, {avg_f:.1f}F, {avg_c:.1f}C")

else:
    print("❌ No lineups were generated successfully.")

print(f"\n✨ TOP {len(top_lineups)} LINEUP GENERATION COMPLETE!")

🎯 EUROLEAGUE FANTASY RULES IMPLEMENTATION
📋 FANTASY RULES:
   • Total Budget: 100.0 credits
   • Total Players: 10
   • 6 Active Players (5 starters + 6th player)
   • 1 Captain (×2.0 points)
   • 4 Substitutes
   • Position Requirements: At least 3G, 3F, 2C
   • Generating Top 5 Lineups
✅ Loaded 280 player projections
✅ 216 valid players for optimization

🔄 Generating Top 5 Lineups...
   Position Requirements: 3G, 3F, 2C
   Generating lineup 1/5...
      ✅ Lineup 1: 1879.42 points, 100.0 credits
         Positions: G:5 F:3 C:2
   Generating lineup 2/5...
      ✅ Lineup 2: 1879.16 points, 99.8 credits
         Positions: G:4 F:4 C:2
   Generating lineup 3/5...
      ✅ Lineup 3: 1875.63 points, 100.0 credits
         Positions: G:5 F:3 C:2
   Generating lineup 4/5...
      ✅ Lineup 4: 1875.37 points, 99.8 credits
         Positions: G:4 F:4 C:2
   Generating lineup 5/5...
      ✅ Lineup 5: 1868.67 points, 99.5 credits
         Positions: G:4 F:4 C:2

TOP 5 EUROLEAGUE FANTASY LINEUPS
POS